<a href="https://colab.research.google.com/github/hssitja/PhD-Dissertation/blob/Chapter-2/Chapter2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# setup and installation

!pip install vaex
!pip install stargazer
!pip install git+https://github.com/statsmodels/statsmodels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 KB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 62.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 KB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 95.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 91.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 4.2 

In [2]:
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

from google.colab import drive, auth
drive.mount('/content/drive')

import os
path = '/content/drive/My Drive/2 Essay Bayesian dinamic model for vaccine hesitancy/Data'
os.chdir(path)

project = 'phd-dissertation-343113' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import vaex
import vaex.ml
from vaex.ml.sklearn import Predictor
import vaex.ml.metrics
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
import statsmodels.api as sm
from statsmodels.tools.tools import pinv_extended
from statsmodels.miscmodels.ordinal_model import OrderedModel

from stargazer.stargazer import Stargazer

SEED = np.random.seed(42)

with open('/content/drive/MyDrive/2 Essay Bayesian dinamic model for vaccine hesitancy/Python/Google developer/config') as f:
    api_key = f.readline()
    f.close

Mounted at /content/drive


In [3]:
city = 'Sample'

df = vaex.open(f'df_{city}.hdf5')

# Filter the rows based on the values in the "id_municipio" column
df = df[df['id_municipio_endereco_paciente'].isin(['1302603', '1506807', '1600303', '2610707', '2611606', '3205309', '3304557', '4106902', '4305108', '4314902', '5103403'])]

# Open sample2 and concatenate
df2 = vaex.open(f'df_{city}2.hdf5')

df = vaex.concat([df, df2])

In [4]:
df.shape

(17926966, 24)

In [5]:
df['votos_PSL_2018'] = df.mean_votos_PSL_2018 / df.mean_votos_tot_2018
df['votos_PT_2018'] = df.mean_votos_PT_2018 / df.mean_votos_tot_2018
df['votos_PSL-PT_2018'] = df.votos_PSL_2018 - df.votos_PT_2018
df.drop(['mean_votos_PSL_2018', 'mean_votos_PT_2018', 'mean_votos_tot_2018'], inplace=True)
df.head(5)

#,id_paciente,sexo_paciente,raca_cor_paciente,cep_endereco_paciente,id_municipio_endereco_paciente,idade_paciente,dose1,dose2,dose3,dose4,nome_fabricante_dose1,nome_fabricante_dose2,nome_fabricante_dose3,nome_fabricante_dose4,grupo_atendimento_dose1,diff_dose1_dose2,diff_dose2_dose3,bairros,mean_income_avg,mean_literacy,mean_families,votos_PSL_2018,votos_PT_2018,votos_PSL-PT_2018
0,'dc434259fdfb193e7b326b49ebb5b60184deaefd8c71968...,M,1,91790,4314902,37,Dose Única,Reforço,--,--,JANSSEN,JANSSEN,--,--,201,157,0,['RESTINGA'],1573.44,96,3.44,0.0587441,0.154866,-0.0961222
1,'ddced77709fb3841c393e71f8295a90b1946353de563be3...,M,99,21616,3304557,41,Dose Única,Reforço,--,--,JANSSEN,JANSSEN,--,--,201,189,0,"['MARECHAL HERMES', 'DEODORO', 'VILA MILITAR']",2222.86,97.8667,3.15333,0.271185,0.0244223,0.246763
2,'ec126e08f8ff3cac6fdeb56d10605651e7a88264d507ce3...,F,2,91320,4314902,47,Dose Única,Dose Adicional,--,--,JANSSEN,JANSSEN,--,--,201,168,0,"['VILA IPIRANGA', 'JARDIM EUROPA', 'VILA JARDIM']",3546.25,98.7,2.685,0.0933469,0.105019,-0.0116725
3,'aed8addd9939018c84adb648b94410201b6e4e4a9003bbf...,F,99,81540,4106902,41,Dose Única,Reforço,--,--,JANSSEN,JANSSEN,--,--,201,162,0,"['UBERABA', 'JARDIM DAS AMÉRICAS']",4628.52,98.5,3.21,0.132806,0.0632321,0.069574
4,'7a922159dd14c956c7278329a41e72c268e962b5271a0a9...,M,99,82110,4106902,48,1ª Dose,Reforço,--,--,JANSSEN,JANSSEN,--,--,1003,172,0,['PILARZINHO'],3506.25,98,3.15,0.127883,0.0637861,0.0640967


In [6]:
[df['dose1'].value_counts(), df['dose2'].value_counts(), df['dose3'].value_counts()]

[1ª Dose           17085724
 2ª Dose             397010
 Dose Única          365818
 Reforço              44286
 Única                24895
 3ª Dose               6852
 Dose Adicional        2324
 1º Reforço              33
 Dose Inicial            18
 4ª Dose                  6
 dtype: int64, missing                 3371925
 2ª Dose                14194019
 Reforço                  221066
 1ª Dose                   96385
 3ª Dose                   28622
 Dose Adicional            13061
 Dose Única                 1544
 Única                       190
 1º Reforço                   71
 4ª Dose                      62
 Dose Inicial                  9
 2ª Dose Revacinação           9
 1ª Dose Revacinação           3
 dtype: int64, missing                14190810
 Reforço                 3017481
 3ª Dose                  395665
 2ª Dose                  170949
 Dose Adicional           128994
 1ª Dose                   21996
 1º Reforço                  586
 Dose Única                  353

In [7]:
df['grupo_atendimento_dose1'].value_counts().nlargest(15)

201    9854377
206    1232081
926    1082317
202     846278
107     609598
203     596723
205     504355
204     411357
801     356086
103     316473
111     177109
104     112354
923     110088
109     104795
301      91366
dtype: int64

In [8]:
df['sexo_paciente'].value_counts(), df['raca_cor_paciente'].value_counts()

(F    9668779
 M    8258155
 I         32
 dtype: int64, 99    5655221
 1     4910279
 3     4830990
 4     1640135
 2      874309
 5       16032
 dtype: int64)

In [9]:
df['nome_fabricante_dose1'].value_counts()

ASTRAZENECA/FIOCRUZ    6280742
PFIZER                 6063334
SINOVAC/BUTANTAN       4745484
JANSSEN                 499404
ASTRAZENECA             338002
dtype: int64

In [10]:
# Creating dummies
one_hot_encoder = vaex.ml.OneHotEncoder(features=['sexo_paciente','raca_cor_paciente','nome_fabricante_dose1','nome_fabricante_dose2','nome_fabricante_dose3'])
df = one_hot_encoder.fit_transform(df)

In [11]:
# Dummies for race:

df['raca_cor_white'] = df['raca_cor_paciente_1']
df['raca_cor_black'] = df['raca_cor_paciente_2']
df['raca_cor_brown'] = df['raca_cor_paciente_3']
df['raca_cor_yellow'] = df['raca_cor_paciente_4']
df['raca_cor_indigenous'] = df['raca_cor_paciente_5']
df.drop(['raca_cor_paciente_1', 'raca_cor_paciente_2', 'raca_cor_paciente_3', 'raca_cor_paciente_4', 'raca_cor_paciente_5'], inplace=True)

# and vaccine supplier:

df['dose1_ASTRAZENECA'] = df['nome_fabricante_dose1_ASTRAZENECA']
df['dose1_JANSSEN'] = df['nome_fabricante_dose1_JANSSEN']
df['dose1_PFIZER'] = df['nome_fabricante_dose1_PFIZER']
df['dose1_SINOVAC'] = df['nome_fabricante_dose1_SINOVAC/BUTANTAN']
df['dose2_ASTRAZENECA'] = df['nome_fabricante_dose2_ASTRAZENECA']
df['dose2_JANSSEN'] = df['nome_fabricante_dose2_JANSSEN']
df['dose2_PFIZER'] = df['nome_fabricante_dose2_PFIZER']
df['dose2_SINOVAC'] = df['nome_fabricante_dose2_SINOVAC/BUTANTAN']
df['dose3_ASTRAZENECA'] = df['nome_fabricante_dose3_ASTRAZENECA']
df['dose3_JANSSEN'] = df['nome_fabricante_dose3_JANSSEN']
df['dose3_PFIZER'] = df['nome_fabricante_dose3_PFIZER']
df['dose3_SINOVAC'] = df['nome_fabricante_dose3_SINOVAC/BUTANTAN']

df.drop(['nome_fabricante_dose1_ASTRAZENECA', 'nome_fabricante_dose1_JANSSEN', 'nome_fabricante_dose1_PFIZER', 'nome_fabricante_dose1_SINOVAC/BUTANTAN',
         'nome_fabricante_dose2_ASTRAZENECA', 'nome_fabricante_dose2_JANSSEN', 'nome_fabricante_dose2_PFIZER', 'nome_fabricante_dose2_SINOVAC/BUTANTAN',
         'nome_fabricante_dose3_ASTRAZENECA', 'nome_fabricante_dose3_JANSSEN', 'nome_fabricante_dose3_PFIZER', 'nome_fabricante_dose3_SINOVAC/BUTANTAN'], inplace=True)
df.head(5)

#,id_paciente,sexo_paciente,raca_cor_paciente,cep_endereco_paciente,id_municipio_endereco_paciente,idade_paciente,dose1,dose2,dose3,dose4,nome_fabricante_dose1,nome_fabricante_dose2,nome_fabricante_dose3,nome_fabricante_dose4,grupo_atendimento_dose1,diff_dose1_dose2,diff_dose2_dose3,bairros,mean_income_avg,mean_literacy,mean_families,votos_PSL_2018,votos_PT_2018,votos_PSL-PT_2018,sexo_paciente_F,sexo_paciente_I,sexo_paciente_M,raca_cor_paciente_99,nome_fabricante_dose1_ASTRAZENECA/FIOCRUZ,nome_fabricante_dose2_missing,nome_fabricante_dose2_ASTRAZENECA/FIOCRUZ,nome_fabricante_dose3_missing,nome_fabricante_dose3_ASTRAZENECA/FIOCRUZ,raca_cor_white,raca_cor_black,raca_cor_brown,raca_cor_yellow,raca_cor_indigenous,dose1_ASTRAZENECA,dose1_JANSSEN,dose1_PFIZER,dose1_SINOVAC,dose2_ASTRAZENECA,dose2_JANSSEN,dose2_PFIZER,dose2_SINOVAC,dose3_ASTRAZENECA,dose3_JANSSEN,dose3_PFIZER,dose3_SINOVAC
0,'dc434259fdfb193e7b326b49ebb5b60184deaefd8c71968...,M,1,91790,4314902,37,Dose Única,Reforço,--,--,JANSSEN,JANSSEN,--,--,201,157,0,['RESTINGA'],1573.44,96,3.44,0.0587441,0.154866,-0.0961222,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
1,'ddced77709fb3841c393e71f8295a90b1946353de563be3...,M,99,21616,3304557,41,Dose Única,Reforço,--,--,JANSSEN,JANSSEN,--,--,201,189,0,"['MARECHAL HERMES', 'DEODORO', 'VILA MILITAR']",2222.86,97.8667,3.15333,0.271185,0.0244223,0.246763,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
2,'ec126e08f8ff3cac6fdeb56d10605651e7a88264d507ce3...,F,2,91320,4314902,47,Dose Única,Dose Adicional,--,--,JANSSEN,JANSSEN,--,--,201,168,0,"['VILA IPIRANGA', 'JARDIM EUROPA', 'VILA JARDIM']",3546.25,98.7,2.685,0.0933469,0.105019,-0.0116725,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
3,'aed8addd9939018c84adb648b94410201b6e4e4a9003bbf...,F,99,81540,4106902,41,Dose Única,Reforço,--,--,JANSSEN,JANSSEN,--,--,201,162,0,"['UBERABA', 'JARDIM DAS AMÉRICAS']",4628.52,98.5,3.21,0.132806,0.0632321,0.069574,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
4,'7a922159dd14c956c7278329a41e72c268e962b5271a0a9...,M,99,82110,4106902,48,1ª Dose,Reforço,--,--,JANSSEN,JANSSEN,--,--,1003,172,0,['PILARZINHO'],3506.25,98,3.15,0.127883,0.0637861,0.0640967,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0


In [12]:
# Creating dummies for priority groups:

# Define a list of age group values for the range [65-80)
age_65_80_values = ['202', '203', '204']

# Create a mask that is True for rows where 'grupo_atendimento_dose1' is in the age range [65-80)
mask_age_65_80 = df['grupo_atendimento_dose1'].isin(age_65_80_values)

# Create a dummy variable 'age_65_80' that is True where 'mask_age_65_80' is True, and False otherwise
df['age_65_80'] = mask_age_65_80.astype('int64')

In [13]:
# Define a list of age group values for the range [>80)
age_80_values = ['205']

# Create a mask that is True for rows where 'grupo_atendimento_dose1' is in the age > 80
mask_80_plus = df['grupo_atendimento_dose1'].isin(age_80_values)

# Create a dummy variable 'age_65_80' that is True where 'mask_80_plus' is True, and False otherwise
df['age_80_plus'] = mask_80_plus.astype('int64')

In [14]:
# Repeating the same process for other characteristics (comorbidities, security forces, health professionals):

comorbidities = {101: 'Sickle cell anemia', 102: 'Neoplasms', 103: 'Diabetes Mellitus', 104: 'Chronic Obstructive Pulmonary Disease',
                 105: 'Chronic Kidney Disease', 106: 'Cardiovascular and cerebrovascular diseases', 107: 'Hypertension with complications or organ damage',
                 108: 'Solid Organ Transplant recipients', 109: 'Severe Obesity (BMI >= 40)', 110: 'Down Syndrome', 111: 'Other Immunocompromised conditions',
                 114: 'Liver Cirrhosis', 115: 'Chronic Neurological Diseases', 116: 'Cardiovascular Disease'}

comorbidities_values = ['106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116']

mask = df['grupo_atendimento_dose1'].astype(str).isin(comorbidities_values)
df['comorbidities'] = mask.astype('int64')

In [15]:
security_forces = {402: 'Brazilian Army - EB', 403: 'Brazilian Air Force - FAB', 501: 'Civil Firefighter', 502: 'Military Firefighter',
                   503: 'Municipal Guard', 505: 'Civil Police', 506: 'Federal Police', 507: 'Military Police'}

security_force_values = ['402', '403', '501', '502', '503', '505', '506', '507']

mask = df['grupo_atendimento_dose1'].astype(str).isin(security_force_values)
df['security_forces'] = mask.astype('int64')


In [16]:
health_professionals = {903: 'Biomedical scientist', 905: 'Elderly caregiver', 906: 'Doula/Midwife', 907: 'Nurse', 908: 'Pharmacist', 909: 'Physiotherapist',
                        910: 'Speech therapist', 911: 'Funeral system employee with potentially contaminated cadavers', 912: 'Doctor', 914: 'Ambulance driver',
                        915: 'Nutritionist', 916: 'Dentist', 919: 'Psychologist', 922: 'Social worker', 923: 'Nursing technician', 925: 'Occupational therapist',
                        927: 'Nursing auxiliary', 928: 'Dental technician', 930: 'Endemic combat agent - ACE', 931: 'Community Health Agent - ACS',
                        932: 'Dental health assistant - ASB', 933: 'Dental health technician - TSB'}

health_professional_values = ['903', '905', '906', '907', '908', '909', '910', '911', '912', '914', '915', '916', '919', '922', '923', '925', '927', '928', '930', '931', '932', '933']

mask = df['grupo_atendimento_dose1'].astype(str).isin(health_professional_values)
df['health_professionals'] = mask.astype('int64')

In [17]:
df['n_doses'] = df['dose1'].notna().astype('int64') + df['dose2'].notna().astype('int64') + df['dose3'].notna().astype('int64') + df['dose4'].notna().astype('int64')

In [18]:
df['n_doses'].value_counts()

2    10818885
3     3679092
1     3371925
4       57064
dtype: int64

In [19]:
df.column_names

['id_paciente',
 'sexo_paciente',
 'raca_cor_paciente',
 'cep_endereco_paciente',
 'id_municipio_endereco_paciente',
 'idade_paciente',
 'dose1',
 'dose2',
 'dose3',
 'dose4',
 'nome_fabricante_dose1',
 'nome_fabricante_dose2',
 'nome_fabricante_dose3',
 'nome_fabricante_dose4',
 'grupo_atendimento_dose1',
 'diff_dose1_dose2',
 'diff_dose2_dose3',
 'bairros',
 '__mean_votos_PSL_2018',
 '__mean_votos_PT_2018',
 '__mean_votos_tot_2018',
 'mean_income_avg',
 'mean_literacy',
 'mean_families',
 'votos_PSL_2018',
 'votos_PT_2018',
 'votos_PSL-PT_2018',
 'sexo_paciente_F',
 'sexo_paciente_I',
 'sexo_paciente_M',
 '__raca_cor_paciente_1',
 '__raca_cor_paciente_2',
 '__raca_cor_paciente_3',
 '__raca_cor_paciente_4',
 '__raca_cor_paciente_5',
 'raca_cor_paciente_99',
 '__nome_fabricante_dose1_ASTRAZENECA',
 'nome_fabricante_dose1_ASTRAZENECA/FIOCRUZ',
 '__nome_fabricante_dose1_JANSSEN',
 '__nome_fabricante_dose1_PFIZER',
 '__nome_fabricante_dose1_SINOVAC/BUTANTAN',
 'nome_fabricante_dose2_missi

In [20]:
dummies = ('sexo_paciente_F','raca_cor_white', 'raca_cor_black', 'raca_cor_brown')

vaccines = ('dose1_ASTRAZENECA', 'nome_fabricante_dose1_ASTRAZENECA/FIOCRUZ', 'dose1_JANSSEN', 'dose1_PFIZER', 'dose1_SINOVAC', 'dose2_ASTRAZENECA', 'nome_fabricante_dose2_ASTRAZENECA/FIOCRUZ', 'dose2_JANSSEN', 'dose2_PFIZER', 'dose2_SINOVAC',
           'dose3_ASTRAZENECA', 'nome_fabricante_dose3_ASTRAZENECA/FIOCRUZ', 'dose3_JANSSEN', 'dose3_PFIZER', 'dose3_SINOVAC')

groups = ('age_65_80', 'age_80_plus', 'comorbidities', 'security_forces', 'health_professionals')

locals = ('id_municipio_endereco_paciente', 'bairros')

features = ('idade_paciente', 'mean_income_avg', 'mean_literacy', 'mean_families', 'votos_PSL-PT_2018')

target = ('diff_dose1_dose2', 'diff_dose2_dose3', 'n_doses')

In [21]:
columns = tuple(list(dummies) + list(features) + list(target) + list(groups))
df.head(5)[columns]

#,sexo_paciente_F,raca_cor_white,raca_cor_black,raca_cor_brown,idade_paciente,mean_income_avg,mean_literacy,mean_families,votos_PSL-PT_2018,diff_dose1_dose2,diff_dose2_dose3,n_doses,age_65_80,age_80_plus,comorbidities,security_forces,health_professionals
0,0,1,0,0,37,1573.44,96,3.44,-0.0961222,157,0,2,0,0,0,0,0
1,0,0,0,0,41,2222.86,97.8667,3.15333,0.246763,189,0,2,0,0,0,0,0
2,1,0,1,0,47,3546.25,98.7,2.685,-0.0116725,168,0,2,0,0,0,0,0
3,1,0,0,0,41,4628.52,98.5,3.21,0.069574,162,0,2,0,0,0,0,0
4,0,0,0,0,48,3506.25,98,3.15,0.0640967,172,0,2,0,0,0,0,0


In [22]:
# Scaling features

scaler = vaex.ml.StandardScaler(features=features,  prefix='scaled_')
scaler.fit(df)

# use a list comprehension to add 'scaled_' to every string in the list
scaled_features = ['scaled_' + feature for feature in features]

df = scaler.transform(df)
df.head(5)

/usr/local/lib/python3.9/dist-packages/vaex/arrow/numpy_dispatch.py:72: RuntimeWarning: invalid value encountered in true_divide
  result_data = op['op'](a_data, b_data)


#,id_paciente,sexo_paciente,raca_cor_paciente,cep_endereco_paciente,id_municipio_endereco_paciente,idade_paciente,dose1,dose2,dose3,dose4,nome_fabricante_dose1,nome_fabricante_dose2,nome_fabricante_dose3,nome_fabricante_dose4,grupo_atendimento_dose1,diff_dose1_dose2,diff_dose2_dose3,bairros,mean_income_avg,mean_literacy,mean_families,votos_PSL_2018,votos_PT_2018,votos_PSL-PT_2018,sexo_paciente_F,sexo_paciente_I,sexo_paciente_M,raca_cor_paciente_99,nome_fabricante_dose1_ASTRAZENECA/FIOCRUZ,nome_fabricante_dose2_missing,nome_fabricante_dose2_ASTRAZENECA/FIOCRUZ,nome_fabricante_dose3_missing,nome_fabricante_dose3_ASTRAZENECA/FIOCRUZ,raca_cor_white,raca_cor_black,raca_cor_brown,raca_cor_yellow,raca_cor_indigenous,dose1_ASTRAZENECA,dose1_JANSSEN,dose1_PFIZER,dose1_SINOVAC,dose2_ASTRAZENECA,dose2_JANSSEN,dose2_PFIZER,dose2_SINOVAC,dose3_ASTRAZENECA,dose3_JANSSEN,dose3_PFIZER,dose3_SINOVAC,age_65_80,age_80_plus,comorbidities,security_forces,health_professionals,n_doses,scaled_idade_paciente,scaled_mean_income_avg,scaled_mean_literacy,scaled_mean_families,scaled_votos_PSL-PT_2018
0,'dc434259fdfb193e7b326b49ebb5b60184deaefd8c71968...,M,1,91790,4314902,37,Dose Única,Reforço,--,--,JANSSEN,JANSSEN,--,--,201,157,0,['RESTINGA'],1573.44,96,3.44,0.0587441,0.154866,-0.0961222,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,-0.290064,-0.737613,-0.169016,0.735782,-1.68253
1,'ddced77709fb3841c393e71f8295a90b1946353de563be3...,M,99,21616,3304557,41,Dose Única,Reforço,--,--,JANSSEN,JANSSEN,--,--,201,189,0,"['MARECHAL HERMES', 'DEODORO', 'VILA MILITAR']",2222.86,97.8667,3.15333,0.271185,0.0244223,0.246763,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,-0.075311,-0.414844,0.567233,0.0316364,1.45415
2,'ec126e08f8ff3cac6fdeb56d10605651e7a88264d507ce3...,F,2,91320,4314902,47,Dose Única,Dose Adicional,--,--,JANSSEN,JANSSEN,--,--,201,168,0,"['VILA IPIRANGA', 'JARDIM EUROPA', 'VILA JARDIM']",3546.25,98.7,2.685,0.0933469,0.105019,-0.0116725,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0.246818,0.242897,0.895916,-1.11874,-0.909994
3,'aed8addd9939018c84adb648b94410201b6e4e4a9003bbf...,F,99,81540,4106902,41,Dose Única,Reforço,--,--,JANSSEN,JANSSEN,--,--,201,162,0,"['UBERABA', 'JARDIM DAS AMÉRICAS']",4628.52,98.5,3.21,0.132806,0.0632321,0.069574,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,-0.075311,0.780796,0.817032,0.170828,-0.166758
4,'7a922159dd14c956c7278329a41e72c268e962b5271a0a9...,M,99,82110,4106902,48,1ª Dose,Reforço,--,--,JANSSEN,JANSSEN,--,--,1003,172,0,['PILARZINHO'],3506.25,98,3.15,0.127883,0.0637861,0.0640967,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0.300507,0.223017,0.619822,0.0234487,-0.216864


In [23]:
columns = tuple(list(target) + list(vaccines) + list(groups) + list(dummies) + list(scaled_features) + list(locals) + list(features))
df = df[columns]
df.head(5)

#,diff_dose1_dose2,diff_dose2_dose3,n_doses,dose1_ASTRAZENECA,nome_fabricante_dose1_ASTRAZENECA/FIOCRUZ,dose1_JANSSEN,dose1_PFIZER,dose1_SINOVAC,dose2_ASTRAZENECA,nome_fabricante_dose2_ASTRAZENECA/FIOCRUZ,dose2_JANSSEN,dose2_PFIZER,dose2_SINOVAC,dose3_ASTRAZENECA,nome_fabricante_dose3_ASTRAZENECA/FIOCRUZ,dose3_JANSSEN,dose3_PFIZER,dose3_SINOVAC,age_65_80,age_80_plus,comorbidities,security_forces,health_professionals,sexo_paciente_F,raca_cor_white,raca_cor_black,raca_cor_brown,scaled_idade_paciente,scaled_mean_income_avg,scaled_mean_literacy,scaled_mean_families,scaled_votos_PSL-PT_2018,id_municipio_endereco_paciente,bairros,idade_paciente,mean_income_avg,mean_literacy,mean_families,votos_PSL-PT_2018
0,157,0,2,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-0.290064,-0.737613,-0.169016,0.735782,-1.68253,4314902,['RESTINGA'],37,1573.44,96,3.44,-0.0961222
1,189,0,2,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.075311,-0.414844,0.567233,0.0316364,1.45415,3304557,"['MARECHAL HERMES', 'DEODORO', 'VILA MILITAR']",41,2222.86,97.8667,3.15333,0.246763
2,168,0,2,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0.246818,0.242897,0.895916,-1.11874,-0.909994,4314902,"['VILA IPIRANGA', 'JARDIM EUROPA', 'VILA JARDIM']",47,3546.25,98.7,2.685,-0.0116725
3,162,0,2,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-0.075311,0.780796,0.817032,0.170828,-0.166758,4106902,"['UBERABA', 'JARDIM DAS AMÉRICAS']",41,4628.52,98.5,3.21,0.069574
4,172,0,2,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.300507,0.223017,0.619822,0.0234487,-0.216864,4106902,['PILARZINHO'],48,3506.25,98,3.15,0.0640967


In [24]:
df = df.to_pandas_df()
df[target[2]]

0           2
1           2
2           2
3           2
4           2
           ..
17926961    2
17926962    2
17926963    2
17926964    2
17926965    2
Name: n_doses, Length: 17926966, dtype: int64

In [25]:
# Define the query
query = """
SELECT 
  m.id_municipio,
  m.id_municipio_tse, 
  m.nome, 
  m.sigla_uf, 
  m.nome_regiao, 
  p.populacao
FROM 
  `basedosdados.br_bd_diretorios_brasil.municipio` AS m
JOIN 
  `basedosdados.br_ibge_populacao.municipio` AS p
ON 
  m.id_municipio = p.id_municipio
JOIN (
  SELECT 
    id_municipio, 
    MAX(ano) AS max_ano 
  FROM 
    `basedosdados.br_ibge_populacao.municipio` 
  GROUP BY 
    id_municipio
) AS max_ano
ON 
  p.id_municipio = max_ano.id_municipio 
  AND p.ano = max_ano.max_ano
"""

# Execute the query and store the results in a dataframe
df3 = client.query(query).to_dataframe()

In [26]:
# Merge the two dataframes on the 'city' column
df = df.merge(df3[['id_municipio','nome_regiao']], left_on='id_municipio_endereco_paciente', right_on='id_municipio')

# Drop the 'nome' column if it is not needed anymore
df = df.drop(columns='id_municipio')

In [27]:
df['dose1_ASTRAZENECA'] = df['dose1_ASTRAZENECA'] | df['nome_fabricante_dose1_ASTRAZENECA/FIOCRUZ']
df.drop('nome_fabricante_dose1_ASTRAZENECA/FIOCRUZ', axis=1, inplace=True)
df['dose2_ASTRAZENECA'] = df['dose2_ASTRAZENECA'] | df['nome_fabricante_dose2_ASTRAZENECA/FIOCRUZ']
df.drop('nome_fabricante_dose2_ASTRAZENECA/FIOCRUZ', axis=1, inplace=True)
df['dose3_ASTRAZENECA'] = df['dose3_ASTRAZENECA'] | df['nome_fabricante_dose3_ASTRAZENECA/FIOCRUZ']
df.drop('nome_fabricante_dose3_ASTRAZENECA/FIOCRUZ', axis=1, inplace=True)

In [28]:
df['dose1_ASTRAZENECA'].value_counts(), df['dose1_JANSSEN'].value_counts()

(0    11308222
 1     6618744
 Name: dose1_ASTRAZENECA, dtype: int64, 0    17427562
 1      499404
 Name: dose1_JANSSEN, dtype: int64)

In [29]:
df['n_doses'].value_counts()

2    10818885
3     3679092
1     3371925
4       57064
Name: n_doses, dtype: int64

In [30]:
# Add one dose when the vaccine was single dose (Janssen)

df.loc[(df['dose1_JANSSEN'] == 1) & (df['n_doses'] == 1), 'n_doses'] = 2
df.loc[(df['dose1_JANSSEN'] == 1) & (df['n_doses'] == 2), 'n_doses'] = 3

In [31]:
df['n_doses'].value_counts()

2    10709369
3     4177442
1     2983091
4       57064
Name: n_doses, dtype: int64

In [32]:
df['id_municipio_endereco_paciente'][df['scaled_votos_PSL-PT_2018'].isna()].value_counts() / df['id_municipio_endereco_paciente'].value_counts(), 

(1302603    0.316982
 1500107    0.050860
 1506807    0.162895
 1600303    0.427323
 2507507    0.464343
 2610707    0.337310
 2611606    0.155071
 2910800    0.352818
 3205309    0.104888
 3304557    0.042843
 3304904    0.069106
 3547809    0.079578
 4106902    0.195875
 4113700    0.444580
 4305108    0.149306
 4314902    0.033504
 5002704    0.183131
 5103403    0.187424
 5107602    0.236520
 5108402    0.570026
 Name: id_municipio_endereco_paciente, dtype: float64,)

In [33]:
df['id_municipio_endereco_paciente'][df['scaled_mean_income_avg'].isna()].value_counts() / df['id_municipio_endereco_paciente'].value_counts()

1302603    0.215212
1500107    0.050860
1506807    0.309025
1600303    0.403914
2507507    0.169337
2610707    0.295210
2611606    0.069759
2910800    0.313966
3205309    0.097869
3304557    0.037047
3304904    0.084686
3547809    0.035059
4106902    0.118027
4113700    0.351265
4305108    0.057728
4314902    0.062042
5002704    0.444859
5103403    0.173625
5107602    0.182354
5108402    0.335890
Name: id_municipio_endereco_paciente, dtype: float64

In [34]:
df['n_doses'] = pd.Categorical(df['n_doses'], categories=[1, 2, 3, 4], ordered=True)

df1 = df[df[target[0]] > 0]
df2 = df[df[target[1]] > 0]

df.shape, df1.shape, df2.shape

((17926966, 37), (14494789, 37), (3672308, 37))

In [35]:
features = 'scaled_idade_paciente', 'scaled_votos_PSL-PT_2018', 'scaled_mean_income_avg'
dummies = 'sexo_paciente_F', 'raca_cor_black', 'raca_cor_brown', 'raca_cor_white', 'nome_regiao'
vaccines = 'dose1_PFIZER', 'dose1_SINOVAC', 'dose1_JANSSEN'

In [36]:
df_descriptive = df.copy()
# df_descriptive.to_csv('df_descriptive.csv', index=False)

In [37]:
df = df.dropna(subset=features, how='any').copy()
df1 = df1.dropna(how='any').copy()
df2 = df2.dropna(how='any').copy()
df.shape, df1.shape, df2.shape

((14495386, 37), (11819859, 37), (3098866, 37))

In [38]:
df = df[list(target) + list(features) + list(dummies) + list(vaccines) + list(groups)] 

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14495386 entries, 0 to 17926965
Data columns (total 19 columns):
 #   Column                    Dtype   
---  ------                    -----   
 0   diff_dose1_dose2          int64   
 1   diff_dose2_dose3          int64   
 2   n_doses                   category
 3   scaled_idade_paciente     float64 
 4   scaled_votos_PSL-PT_2018  float64 
 5   scaled_mean_income_avg    float64 
 6   sexo_paciente_F           uint8   
 7   raca_cor_black            uint8   
 8   raca_cor_brown            uint8   
 9   raca_cor_white            uint8   
 10  nome_regiao               object  
 11  dose1_PFIZER              uint8   
 12  dose1_SINOVAC             uint8   
 13  dose1_JANSSEN             uint8   
 14  age_65_80                 int64   
 15  age_80_plus               int64   
 16  comorbidities             int64   
 17  security_forces           int64   
 18  health_professionals      int64   
dtypes: category(1), float64(3), int64(7), ob

In [39]:
# Visualizing correlation between features:
corr = df.corr(method='pearson')
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
corr[mask] = np.nan
(corr
 .style
 .background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)
 .highlight_null(null_color='#f1f1f1')  # Color NaNs grey
 .set_precision(2))

<ipython-input-39-0c4efef7cc85>:6: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  (corr


,diff_dose1_dose2,diff_dose2_dose3,scaled_idade_paciente,scaled_votos_PSL-PT_2018,scaled_mean_income_avg,sexo_paciente_F,raca_cor_black,raca_cor_brown,raca_cor_white,dose1_PFIZER,dose1_SINOVAC,dose1_JANSSEN,age_65_80,age_80_plus,comorbidities,security_forces,health_professionals
diff_dose1_dose2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
diff_dose2_dose3,-0.10,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
scaled_idade_paciente,0.08,0.48,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
scaled_votos_PSL-PT_2018,0.06,0.05,0.05,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
scaled_mean_income_avg,0.03,0.13,0.12,0.09,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
sexo_paciente_F,0.02,0.07,0.06,-0.01,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
raca_cor_black,-0.00,-0.00,0.00,0.02,-0.06,0.01,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
raca_cor_brown,-0.01,-0.07,-0.08,0.03,-0.18,0.02,-0.14,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
raca_cor_white,0.01,0.07,0.04,-0.02,0.04,0.05,-0.14,-0.37,nan,nan,nan,nan,nan,nan,nan,nan,nan
dose1_PFIZER,-0.01,-0.30,-0.48,-0.02,-0.05,-0.03,-0.00,0.05,-0.01,nan,nan,nan,nan,nan,nan,nan,nan


In [40]:
# define a variable to store a list of models:

models = []

In [41]:
df = df.rename(columns={'sexo_paciente_F':'Female','raca_cor_black':'Black','raca_cor_brown':'Brown',
                        'raca_cor_white':'White','dose1_PFIZER':'Dose1 Pfizer', 'dose1_SINOVAC':'Dose1 SINOVAC',
                        'scaled_idade_paciente':'Age','scaled_votos_PSL-PT_2018':'Vote diff. PSL-PT',
                        'scaled_mean_income_avg':'Avg. Income'})

In [42]:
dummies = 'Female', 'Black', 'Brown', 'White', 'Dose1 Pfizer', 'Dose1 SINOVAC'
dummies2 = 'Female', 'Black', 'Brown', 'White'
features = 'Age', 'Vote diff. PSL-PT', 'Avg. Income'

-------------------------------------------------------------------------------
General Population - without Janssen

In [43]:
vaccine = 'dose1_JANSSEN'

df1 = df[df[target[0]] > 0]
df2 = df[df[target[1]] > 0]
df1 = df1.dropna(how='any').copy()
df2 = df2.dropna(how='any').copy()

df3 = df[df[vaccine] == 0]
df1 = df1[df1[vaccine] == 0]
df2 = df2[df2[vaccine] == 0]


# setting DF for OLS 1 & 2
X1 = sm.add_constant(df1[list(features) + list(dummies)])
X2 = sm.add_constant(df2[list(features) + list(dummies)])

# setting DF for logit (X3)
X3 = df3[list(features)+ list(dummies)]

# target1 (y1) as diff_dose1_dose2
y1 = df1[target[0]].values.reshape(-1, 1)

# target2 (y2) as diff_dose2_dose3
y2 = df2[target[1]].values.reshape(-1, 1)

# and target3 (y3) as n_doses
y3 = df3[target[2]]

In [44]:
# OLS 1 - diff_dose1_dose2 ~ features + dummies

model_n = '1x'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y1, X1).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.420
Model:                            OLS   Adj. R-squared:                  0.420
Method:                 Least Squares   F-statistic:                 9.431e+05
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        13:30:15   Log-Likelihood:            -5.5030e+07
No. Observations:            11733790   AIC:                         1.101e+08
Df Residuals:                11733780   BIC:                         1.101e+08
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                86.7336      0.017   5007.216      0.000      86.700      86.768
Age                   1.5611      0.009    176.973      0.000       1.544       1.578
Vote diff. PSL-PT     1.1245      0.008    144.667      0.000       1.109       1.140
Avg. Income          -1.4207      0.008   -183.099      0.000      -1.436      -1.405
Female                0.1026      0.016      6.604      0.000       0.072       0.133
Black                 1.5227      0.036     41.939      0.000       1.452       1.594
Brown                 1.0284      0.020     51.932      0.000       0.990       1.067
White                -0.9443      0.019    -49.883      0.000      -0.981      -0.907
Dose1 Pfizer         -5.5894      0.020   -275.431      0.000      -5.629      -5.550
Dose1 SINOVAC       -51.5551      0.019  -2767.722      0.000     -51.592     -51.519
================================================================================
Omnibus:                 54134599.975   Durbin-Watson:                     1.839
Prob(Omnibus):                  0.000   Jarque-Bera (JB):   6366481591071008.000
Skew:                         161.852   Prob(JB):                           0.00
Kurtosis:                  114115.826   Cond. No.                           6.49
================================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [45]:
# OLS 2 - diff_dose2_dose3 ~ features + dummies

model_n = '2x'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y2, X2).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.385
Model:                            OLS   Adj. R-squared:                  0.385
Method:                 Least Squares   F-statistic:                 2.155e+05
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        13:30:17   Log-Likelihood:            -1.5179e+07
No. Observations:             3098474   AIC:                         3.036e+07
Df Residuals:                 3098464   BIC:                         3.036e+07
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               139.5723      0.047   2946.084      0.000     139.479     139.665
Age                   1.4137      0.022     63.058      0.000       1.370       1.458
Vote diff. PSL-PT    -7.3928      0.020   -377.837      0.000      -7.431      -7.354
Avg. Income           0.2236      0.017     13.296      0.000       0.191       0.257
Female                4.3520      0.038    114.934      0.000       4.278       4.426
Black                -1.5433      0.087    -17.812      0.000      -1.713      -1.374
Brown                -1.1231      0.051    -21.943      0.000      -1.223      -1.023
White                -0.5928      0.044    -13.521      0.000      -0.679      -0.507
Dose1 Pfizer        -14.5105      0.079   -183.642      0.000     -14.665     -14.356
Dose1 SINOVAC        44.9757      0.040   1137.883      0.000      44.898      45.053
==============================================================================
Omnibus:                   463483.832   Durbin-Watson:                   1.834
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2861890.579
Skew:                          -0.574   Prob(JB):                         0.00
Kurtosis:                       7.566   Cond. No.                         8.87
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [46]:
# setting DF for OLS 1 & 2 minuns dummies for vaccines
X1 = sm.add_constant(df1[list(features) + list(dummies2)])
X2 = sm.add_constant(df2[list(features) + list(dummies2)])


In [47]:
# OLS 1 - diff_dose1_dose2 ~ features + dummies minuns dummies for vaccines

model_n = '1x2'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y1, X1).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                 2.616e+04
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        13:30:24   Log-Likelihood:            -5.8133e+07
No. Observations:            11733790   AIC:                         1.163e+08
Df Residuals:                11733782   BIC:                         1.163e+08
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                71.4130      0.019   3745.788      0.000      71.376      71.450
Age                  -3.5469      0.010   -349.081      0.000      -3.567      -3.527
Vote diff. PSL-PT     0.9886      0.010     97.650      0.000       0.969       1.008
Avg. Income          -1.5135      0.010   -149.790      0.000      -1.533      -1.494
Female               -0.5477      0.020    -27.059      0.000      -0.587      -0.508
Black                 0.3047      0.047      6.443      0.000       0.212       0.397
Brown                 0.5824      0.026     22.580      0.000       0.532       0.633
White                -2.0267      0.025    -82.235      0.000      -2.075      -1.978
===============================================================================
Omnibus:                 42327756.187   Durbin-Watson:                    1.037
Prob(Omnibus):                  0.000   Jarque-Bera (JB):   767705856110271.625
Skew:                          72.639   Prob(JB):                          0.00
Kurtosis:                   39629.067   Cond. No.                          6.19
===============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [48]:
# OLS 2 - diff_dose2_dose3 ~ features + dummies minuns dummies for vaccines

model_n = '2x2'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y2, X2).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.085
Model:                            OLS   Adj. R-squared:                  0.085
Method:                 Least Squares   F-statistic:                 4.123e+04
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        13:30:25   Log-Likelihood:            -1.5794e+07
No. Observations:             3098474   AIC:                         3.159e+07
Df Residuals:                 3098466   BIC:                         3.159e+07
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               154.8585      0.052   2961.568      0.000     154.756     154.961
Age                   9.2024      0.026    354.538      0.000       9.152       9.253
Vote diff. PSL-PT    -9.2250      0.024   -387.963      0.000      -9.272      -9.178
Avg. Income          -0.0103      0.021     -0.502      0.615      -0.050       0.030
Female                5.9373      0.046    128.644      0.000       5.847       6.028
Black                -2.5649      0.106    -24.274      0.000      -2.772      -2.358
Brown                -1.6028      0.062    -25.679      0.000      -1.725      -1.480
White                -0.8358      0.053    -15.634      0.000      -0.941      -0.731
==============================================================================
Omnibus:                   142022.048   Durbin-Watson:                   1.289
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           502374.419
Skew:                           0.051   Prob(JB):                         0.00
Kurtosis:                       4.970   Cond. No.                         8.41
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [49]:
# Ordered Logit - n_doses ~ features + dummies

model_n = '3x'
model_name = f"model{model_n}"

# Create the Logit model with a different name
model = OrderedModel(y3,X3,distr='logit').fit(method='bfgs', maxiter=2000)

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

Optimization terminated successfully.
         Current function value: 0.788524
         Iterations: 48
         Function evaluations: 49
         Gradient evaluations: 49


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:                n_doses   Log-Likelihood:            -1.1116e+07
Model:                   OrderedModel   AIC:                         2.223e+07
Method:            Maximum Likelihood   BIC:                         2.223e+07
Date:                Sun, 26 Mar 2023                                         
Time:                        14:15:57                                         
No. Observations:            14097711                                         
Df Residuals:                14097699                                         
Df Model:                           9                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Age                   0.9083      0.001   1278.267      0.000       0.907       0.910
Vote diff. PSL-PT     0.1585      0.001    281.739      0.000       0.157       0.160
Avg. Income           0.2186      0.001    371.887      0.000       0.217       0.220
Female                0.1916      0.001    169.125      0.000       0.189       0.194
Black                 0.0475      0.003     17.997      0.000       0.042       0.053
Brown                -0.0343      0.001    -24.091      0.000      -0.037      -0.031
White                 0.3082      0.001    219.790      0.000       0.305       0.311
Dose1 Pfizer         -0.5908      0.002   -389.908      0.000      -0.594      -0.588
Dose1 SINOVAC         0.6357      0.001    442.952      0.000       0.633       0.639
1/2                  -1.9634      0.001  -1346.135      0.000      -1.966      -1.961
2/3                   1.3193      0.000   4025.793      0.000       1.319       1.320
3/4                   1.5935      0.001   1714.132      0.000       1.592       1.595
=====================================================================================
"""

In [50]:
model3x.prsquared

0.16974851910720945

In [51]:
table = Stargazer([model3x, model1x, model1x2, model2x, model2x2])
print(table.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) \\
\hline \\[-1.8ex]
 1/2 & -1.963$^{***}$ & & & & \\
  & (0.001) & & & & \\
 2/3 & 1.319$^{***}$ & & & & \\
  & (0.000) & & & & \\
 3/4 & 1.593$^{***}$ & & & & \\
  & (0.001) & & & & \\
 Age & 0.908$^{***}$ & 1.561$^{***}$ & -3.547$^{***}$ & 1.414$^{***}$ & 9.202$^{***}$ \\
  & (0.001) & (0.009) & (0.010) & (0.022) & (0.026) \\
 Avg. Income & 0.219$^{***}$ & -1.421$^{***}$ & -1.513$^{***}$ & 0.224$^{***}$ & -0.010$^{}$ \\
  & (0.001) & (0.008) & (0.010) & (0.017) & (0.021) \\
 Black & 0.047$^{***}$ & 1.523$^{***}$ & 0.305$^{***}$ & -1.543$^{***}$ & -2.565$^{***}$ \\
  & (0.003) & (0.036) & (0.047) & (0.087) & (0.106) \\
 Brown & -0.034$^{***}$ & 1.028$^{***}$ & 0.582$^{***}$ & -1.123$^{***}$ & -1.603$^{***}$ \\
  & (0.001) & (0.020) & (0.026) & (0.051) & (0.062) \\
 Dose1 Pfizer & -0.591$^{***}$ & -5.589$^{***}$ & & -14.510$^{***}$ & \

--------------------------------------------------------------------
General Population - only Janssen as dose1

In [52]:
# vaccine = 'dose1_JANSSEN'

# df1 = df[df[target[0]] > 0]
# df2 = df[df[target[1]] > 0]
# df1 = df1.dropna(how='any').copy()
# df2 = df2.dropna(how='any').copy()

# df3 = df[df[vaccine] == 1]
# df1 = df1[df1[vaccine] == 1]
# df2 = df2[df2[vaccine] == 1]


# # setting DF for OLS 1 & 2
# X1 = sm.add_constant(df1[list(features) + list(dummies2)])
# X2 = sm.add_constant(df2[list(features) + list(dummies2)])

# # setting DF for logit (X3)
# X3 = df3[list(features)+ list(dummies2)]

# # target1 (y1) as diff_dose1_dose2
# y1 = df1[target[0]].values.reshape(-1, 1)

# # target2 (y2) as diff_dose2_dose3
# y2 = df2[target[1]].values.reshape(-1, 1)

# # and target3 (y3) as n_doses
# y3 = df3[target[2]]

In [53]:
# # OLS 1 - diff_dose1_dose2 ~ features + dummies

# model_n = '1y'
# model_name = f"model{model_n}"

# # Create the OLS model with a different name
# model = sm.OLS(y1, X1).fit()

# # Set the name of the variable dynamically using exec
# exec(f"{model_name} = model")

# # Add the model name to the list of models
# models.append(model_name)

# model.summary()

In [54]:
# # OLS 2 - diff_dose2_dose3 ~ features + dummies

# model_n = '2y'
# model_name = f"model{model_n}"

# # Create the OLS model with a different name
# model = sm.OLS(y2, X2).fit()

# # Set the name of the variable dynamically using exec
# exec(f"{model_name} = model")

# # Add the model name to the list of models
# models.append(model_name)

# model.summary()

In [55]:
# # Ordered Logit - n_doses ~ features + dummies

# model_n = '3y'
# model_name = f"model{model_n}"

# # Create the Logit model with a different name
# model = OrderedModel(y3,X3,distr='logit').fit(method='bfgs', maxiter=2000)

# # Set the name of the variable dynamically using exec
# exec(f"{model_name} = model")

# # Add the model name to the list of models
# models.append(model_name)

# model.summary()

In [56]:
# table = Stargazer([model1y, model2y, model3y])
# # table = Stargazer([model1y, model2y])
# print(table.render_latex())


--------------------------------------------------------------------------------
General Population



In [57]:
# # setting DF for OLS 1 & 2
# X1 = sm.add_constant(df1[list(features) + list(dummies)])
# X2 = sm.add_constant(df2[list(features) + list(dummies)])

# # setting DF for logit (X3)
# X3 = df[list(features)+ list(dummies)]

# # target1 (y1) as diff_dose1_dose2
# y1 = df1[target[0]].values.reshape(-1, 1)

# # target2 (y2) as diff_dose2_dose3
# y2 = df2[target[1]].values.reshape(-1, 1)

# # and target3 (y3) as n_doses
# y3 = df[target[2]]

In [58]:
# # OLS 1 - diff_dose1_dose2 ~ features + dummies

# model_n = 1
# model_name = f"model{model_n}"

# # Create the OLS model with a different name
# model = sm.OLS(y1, X1).fit()

# # Set the name of the variable dynamically using exec
# exec(f"{model_name} = model")

# # Add the model name to the list of models
# models.append(model_name)

# model.summary()

In [59]:
# # OLS 2 - diff_dose2_dose3 ~ features + dummies

# model_n = 2
# model_name = f"model{model_n}"

# # Create the OLS model with a different name
# model = sm.OLS(y2, X2).fit()

# # Set the name of the variable dynamically using exec
# exec(f"{model_name} = model")

# # Add the model name to the list of models
# models.append(model_name)

# model.summary()

In [60]:
# # Ordered Logit - n_doses ~ features + dummies

# model_n = 3
# model_name = f"model{model_n}"

# # Create the Logit model with a different name
# model = OrderedModel(y3,X3,distr='probit').fit(method='bfgs', maxiter=2000)

# # Set the name of the variable dynamically using exec
# exec(f"{model_name} = model")

# # Add the model name to the list of models
# models.append(model_name)

# model.summary()

In [61]:
# model3.prsquared

In [62]:
# table = Stargazer([model1, model2, model3])

In [63]:
# print(table.render_latex())

-------------------------------------------------------------------------------------

Population with comorbidities


In [64]:
# Defining population

group = 'comorbidities'
vaccine = 'dose1_JANSSEN'

df1 = df[df[target[0]] > 0]
df2 = df[df[target[1]] > 0]
df1 = df1.dropna(how='any').copy()
df2 = df2.dropna(how='any').copy()

df3 = df[(df[vaccine] == 0) & (df[group] == 1)]
df1 = df1[(df1[vaccine] == 0) & (df1[group] == 1)]
df2 = df2[(df2[vaccine] == 0) & (df2[group] == 1)]

# setting DF for OLS 1 & 2
X1 = sm.add_constant(df1[list(features) + list(dummies)])
X2 = sm.add_constant(df2[list(features) + list(dummies)])

# setting DF for logit (X3)
X3 = df3[list(features)+ list(dummies)]

# target1 (y1) as diff_dose1_dose2
y1 = df1[target[0]].values.reshape(-1, 1)

# target2 (y2) as diff_dose2_dose3
y2 = df2[target[1]].values.reshape(-1, 1)

# and target3 (y3) as n_doses
y3 = df3[target[2]]

In [65]:
# OLS 1 - diff_dose1_dose2 ~ features + dummies

model_n = '1a'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y1, X1).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.124
Model:                            OLS   Adj. R-squared:                  0.124
Method:                 Least Squares   F-statistic:                 1.233e+04
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:16:05   Log-Likelihood:            -3.8250e+06
No. Observations:              786501   AIC:                         7.650e+06
Df Residuals:                  786491   BIC:                         7.650e+06
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                86.3099      0.073   1174.837      0.000      86.166      86.454
Age                  -0.9451      0.063    -15.093      0.000      -1.068      -0.822
Vote diff. PSL-PT     2.3032      0.037     62.349      0.000       2.231       2.376
Avg. Income          -0.9837      0.036    -27.172      0.000      -1.055      -0.913
Female                0.4523      0.072      6.314      0.000       0.312       0.593
Black                 0.2509      0.152      1.646      0.100      -0.048       0.550
Brown                 0.7072      0.092      7.653      0.000       0.526       0.888
White                -1.4717      0.087    -16.878      0.000      -1.643      -1.301
Dose1 Pfizer         -2.8244      0.085    -33.073      0.000      -2.992      -2.657
Dose1 SINOVAC       -51.8169      0.167   -310.540      0.000     -52.144     -51.490
================================================================================
Omnibus:                  4647411.222   Durbin-Watson:                     1.869
Prob(Omnibus):                  0.000   Jarque-Bera (JB):   2119803634285079.000
Skew:                         398.527   Prob(JB):                           0.00
Kurtosis:                  254335.386   Cond. No.                           6.27
================================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [66]:
# OLS 2 - diff_dose2_dose3 ~ features + dummies

model_n = '2a'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y2, X2).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.162
Model:                            OLS   Adj. R-squared:                  0.162
Method:                 Least Squares   F-statistic:                     5157.
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:16:05   Log-Likelihood:            -1.1585e+06
No. Observations:              239676   AIC:                         2.317e+06
Df Residuals:                  239666   BIC:                         2.317e+06
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               130.7285      0.138    945.431      0.000     130.457     131.000
Age                   9.5182      0.117     81.168      0.000       9.288       9.748
Vote diff. PSL-PT    -1.2225      0.068    -17.883      0.000      -1.357      -1.089
Avg. Income          -1.7852      0.063    -28.191      0.000      -1.909      -1.661
Female                1.6643      0.126     13.223      0.000       1.418       1.911
Black                 3.9247      0.265     14.796      0.000       3.405       4.445
Brown                 1.7926      0.171     10.467      0.000       1.457       2.128
White                 0.4388      0.152      2.886      0.004       0.141       0.737
Dose1 Pfizer         -1.0662      0.147     -7.267      0.000      -1.354      -0.779
Dose1 SINOVAC        45.9196      0.260    176.566      0.000      45.410      46.429
==============================================================================
Omnibus:                    66933.219   Durbin-Watson:                   1.750
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           174141.025
Skew:                          -1.523   Prob(JB):                         0.00
Kurtosis:                       5.857   Cond. No.                         6.33
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [67]:
# setting DF for OLS 1 & 2 minuns dummies for vaccines
X1 = sm.add_constant(df1[list(features) + list(dummies2)])
X2 = sm.add_constant(df2[list(features) + list(dummies2)])


In [68]:
# OLS 1 - diff_dose1_dose2 ~ features + dummies minuns dummies for vaccines

model_n = '1a2'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y1, X1).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     1843.
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:16:06   Log-Likelihood:            -3.8705e+06
No. Observations:              786501   AIC:                         7.741e+06
Df Residuals:                  786493   BIC:                         7.741e+06
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                82.7082      0.074   1121.448      0.000      82.564      82.853
Age                  -0.9698      0.066    -14.625      0.000      -1.100      -0.840
Vote diff. PSL-PT     4.2341      0.039    109.784      0.000       4.158       4.310
Avg. Income          -0.8520      0.038    -22.264      0.000      -0.927      -0.777
Female                0.5980      0.076      7.879      0.000       0.449       0.747
Black                 1.8633      0.161     11.546      0.000       1.547       2.180
Brown                -1.1636      0.098    -11.912      0.000      -1.355      -0.972
White                -0.0598      0.092     -0.649      0.517      -0.241       0.121
================================================================================
Omnibus:                  4440188.779   Durbin-Watson:                     1.669
Prob(Omnibus):                  0.000   Jarque-Bera (JB):   1337654607546344.500
Skew:                         335.141   Prob(JB):                           0.00
Kurtosis:                  202037.523   Cond. No.                           5.87
================================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [69]:
# OLS 2 - diff_dose2_dose3 ~ features + dummies minuns dummies for vaccines

model_n = '2a2'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y2, X2).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.049
Model:                            OLS   Adj. R-squared:                  0.049
Method:                 Least Squares   F-statistic:                     1749.
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:16:06   Log-Likelihood:            -1.1738e+06
No. Observations:              239676   AIC:                         2.348e+06
Df Residuals:                  239668   BIC:                         2.348e+06
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               134.2394      0.138    970.224      0.000     133.968     134.511
Age                  10.0505      0.125     80.479      0.000       9.806      10.295
Vote diff. PSL-PT    -3.9723      0.071    -56.150      0.000      -4.111      -3.834
Avg. Income          -2.2150      0.067    -32.964      0.000      -2.347      -2.083
Female                1.5125      0.134     11.278      0.000       1.250       1.775
Black                 1.6886      0.282      5.986      0.000       1.136       2.241
Brown                 4.6523      0.182     25.630      0.000       4.297       5.008
White                -1.8350      0.161    -11.371      0.000      -2.151      -1.519
==============================================================================
Omnibus:                    49331.855   Durbin-Watson:                   1.561
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           109749.705
Skew:                          -1.186   Prob(JB):                         0.00
Kurtosis:                       5.316   Cond. No.                         6.11
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [70]:
# Ordered Logit - n_doses ~ features + dummies

model_n = '3a'
model_name = f"model{model_n}"

# Create the Logit model with a different name
model = OrderedModel(y3,X3,distr='logit').fit(method='bfgs', maxiter=2000)

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

Optimization terminated successfully.
         Current function value: 0.831661
         Iterations: 43
         Function evaluations: 44
         Gradient evaluations: 44


In [71]:
model3a.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:                n_doses   Log-Likelihood:            -7.0227e+05
Model:                   OrderedModel   AIC:                         1.405e+06
Method:            Maximum Likelihood   BIC:                         1.405e+06
Date:                Sun, 26 Mar 2023                                         
Time:                        14:19:01                                         
No. Observations:              844424                                         
Df Residuals:                  844412                                         
Df Model:                           9                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Age                   0.4450      0.004    109.846      0.000       0.437       0.453
Vote diff. PSL-PT     0.1596      0.002     67.462      0.000       0.155       0.164
Avg. Income           0.1218      0.002     53.350      0.000       0.117       0.126
Female               -0.0538      0.005    -11.766      0.000      -0.063      -0.045
Black                 0.2468      0.010     25.425      0.000       0.228       0.266
Brown                 0.0105      0.006      1.778      0.075      -0.001       0.022
White                 0.4037      0.006     72.522      0.000       0.393       0.415
Dose1 Pfizer          0.0738      0.005     13.574      0.000       0.063       0.084
Dose1 SINOVAC         0.6968      0.010     66.460      0.000       0.676       0.717
1/2                  -2.4455      0.006   -415.215      0.000      -2.457      -2.434
2/3                   1.2918      0.001    984.316      0.000       1.289       1.294
3/4                   1.4769      0.003    433.304      0.000       1.470       1.484
=====================================================================================
"""

In [72]:
model3a.prsquared

0.01951833315220075

In [73]:
table = Stargazer([model3a, model1a, model1a2, model2a, model2a2])

In [74]:
print(table.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) \\
\hline \\[-1.8ex]
 1/2 & -2.446$^{***}$ & & & & \\
  & (0.006) & & & & \\
 2/3 & 1.292$^{***}$ & & & & \\
  & (0.001) & & & & \\
 3/4 & 1.477$^{***}$ & & & & \\
  & (0.003) & & & & \\
 Age & 0.445$^{***}$ & -0.945$^{***}$ & -0.970$^{***}$ & 9.518$^{***}$ & 10.050$^{***}$ \\
  & (0.004) & (0.063) & (0.066) & (0.117) & (0.125) \\
 Avg. Income & 0.122$^{***}$ & -0.984$^{***}$ & -0.852$^{***}$ & -1.785$^{***}$ & -2.215$^{***}$ \\
  & (0.002) & (0.036) & (0.038) & (0.063) & (0.067) \\
 Black & 0.247$^{***}$ & 0.251$^{*}$ & 1.863$^{***}$ & 3.925$^{***}$ & 1.689$^{***}$ \\
  & (0.010) & (0.152) & (0.161) & (0.265) & (0.282) \\
 Brown & 0.011$^{*}$ & 0.707$^{***}$ & -1.164$^{***}$ & 1.793$^{***}$ & 4.652$^{***}$ \\
  & (0.006) & (0.092) & (0.098) & (0.171) & (0.182) \\
 Dose1 Pfizer & 0.074$^{***}$ & -2.824$^{***}$ & & -1.066$^{***}$ & \\
  

-------------------------------------------------------------------------------------------------
Health Professionals

In [75]:
# Defining population

group = 'health_professionals'
vaccine = 'dose1_JANSSEN'

df1 = df[df[target[0]] > 0]
df2 = df[df[target[1]] > 0]
df1 = df1.dropna(how='any').copy()
df2 = df2.dropna(how='any').copy()

df3 = df[(df[vaccine] == 0) & (df[group] == 1)]
df1 = df1[(df1[vaccine] == 0) & (df1[group] == 1)]
df2 = df2[(df2[vaccine] == 0) & (df2[group] == 1)]

# setting DF for OLS 1 & 2
X1 = sm.add_constant(df1[list(features) + list(dummies)])
X2 = sm.add_constant(df2[list(features) + list(dummies)])

# setting DF for logit (X3)
X3 = df3[list(features)+ list(dummies)]

# target1 (y1) as diff_dose1_dose2
y1 = df1[target[0]].values.reshape(-1, 1)

# target2 (y2) as diff_dose2_dose3
y2 = df2[target[1]].values.reshape(-1, 1)

# and target3 (y3) as n_doses
y3 = df3[target[2]]

In [76]:
# OLS 1 - diff_dose1_dose2 ~ features + dummies

model_n = '1a'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y1, X1).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.425
Model:                            OLS   Adj. R-squared:                  0.425
Method:                 Least Squares   F-statistic:                 3.016e+04
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:19:06   Log-Likelihood:            -1.7865e+06
No. Observations:              366609   AIC:                         3.573e+06
Df Residuals:                  366599   BIC:                         3.573e+06
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                88.8706      0.141    632.389      0.000      88.595      89.146
Age                   0.4724      0.077      6.147      0.000       0.322       0.623
Vote diff. PSL-PT    -0.3457      0.055     -6.307      0.000      -0.453      -0.238
Avg. Income          -0.3677      0.044     -8.384      0.000      -0.454      -0.282
Female                0.2991      0.124      2.408      0.016       0.056       0.542
Black                 2.9490      0.272     10.846      0.000       2.416       3.482
Brown                 0.5358      0.150      3.581      0.000       0.243       0.829
White                 0.5559      0.122      4.566      0.000       0.317       0.795
Dose1 Pfizer         -2.5145      0.259     -9.715      0.000      -3.022      -2.007
Dose1 SINOVAC       -54.9813      0.112   -492.983      0.000     -55.200     -54.763
==============================================================================
Omnibus:                   386826.337   Durbin-Watson:                   1.387
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         19782376.845
Skew:                           5.551   Prob(JB):                         0.00
Kurtosis:                      37.232   Cond. No.                         8.59
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [77]:
# OLS 2 - diff_dose2_dose3 ~ features + dummies

model_n = '2a'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y2, X2).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.417
Model:                            OLS   Adj. R-squared:                  0.417
Method:                 Least Squares   F-statistic:                 1.776e+04
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:19:06   Log-Likelihood:            -1.0945e+06
No. Observations:              223066   AIC:                         2.189e+06
Df Residuals:                  223056   BIC:                         2.189e+06
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               174.6307      0.197    886.727      0.000     174.245     175.017
Age                  -1.3111      0.100    -13.127      0.000      -1.507      -1.115
Vote diff. PSL-PT    -0.5106      0.073     -6.956      0.000      -0.654      -0.367
Avg. Income          -1.2589      0.057    -21.905      0.000      -1.372      -1.146
Female               -1.1085      0.164     -6.758      0.000      -1.430      -0.787
Black                 1.1458      0.369      3.106      0.002       0.423       1.869
Brown                 0.0364      0.207      0.176      0.860      -0.369       0.441
White                -1.0877      0.159     -6.826      0.000      -1.400      -0.775
Dose1 Pfizer        -29.0826      0.501    -58.033      0.000     -30.065     -28.100
Dose1 SINOVAC        54.2979      0.152    356.453      0.000      53.999      54.596
==============================================================================
Omnibus:                    85532.142   Durbin-Watson:                   1.831
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           798116.173
Skew:                          -1.587   Prob(JB):                         0.00
Kurtosis:                      11.706   Cond. No.                         12.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [78]:
# setting DF for OLS 1 & 2 minuns dummies for vaccines
X1 = sm.add_constant(df1[list(features) + list(dummies2)])
X2 = sm.add_constant(df2[list(features) + list(dummies2)])


In [79]:
# OLS 1 - diff_dose1_dose2 ~ features + dummies minuns dummies for vaccines

model_n = '1a2'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y1, X1).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     1528.
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:19:06   Log-Likelihood:            -1.8828e+06
No. Observations:              366609   AIC:                         3.766e+06
Df Residuals:                  366601   BIC:                         3.766e+06
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                59.3802      0.164    363.155      0.000      59.060      59.701
Age                   1.1748      0.099     11.829      0.000       0.980       1.369
Vote diff. PSL-PT     6.8205      0.069     99.170      0.000       6.686       6.955
Avg. Income           0.4037      0.057      7.083      0.000       0.292       0.515
Female                1.2987      0.161      8.045      0.000       0.982       1.615
Black                 0.4087      0.354      1.156      0.248      -0.284       1.102
Brown                -2.5049      0.194    -12.885      0.000      -2.886      -2.124
White                -1.4784      0.158     -9.343      0.000      -1.789      -1.168
==============================================================================
Omnibus:                   168928.019   Durbin-Watson:                   0.712
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1244190.865
Skew:                           2.086   Prob(JB):                         0.00
Kurtosis:                      11.003   Cond. No.                         8.24
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [80]:
# OLS 2 - diff_dose2_dose3 ~ features + dummies minuns dummies for vaccines

model_n = '2a2'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y2, X2).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     1351.
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:19:06   Log-Likelihood:            -1.1502e+06
No. Observations:              223066   AIC:                         2.300e+06
Df Residuals:                  223058   BIC:                         2.300e+06
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               209.1054      0.217    964.196      0.000     208.680     209.530
Age                  -5.7164      0.126    -45.285      0.000      -5.964      -5.469
Vote diff. PSL-PT    -5.9549      0.092    -64.466      0.000      -6.136      -5.774
Avg. Income          -2.3256      0.074    -31.582      0.000      -2.470      -2.181
Female               -2.1525      0.210    -10.228      0.000      -2.565      -1.740
Black                 3.5805      0.473      7.566      0.000       2.653       4.508
Brown                 3.5328      0.265     13.340      0.000       3.014       4.052
White                 0.4302      0.204      2.105      0.035       0.030       0.831
==============================================================================
Omnibus:                    25283.569   Durbin-Watson:                   1.143
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            57922.343
Skew:                          -0.687   Prob(JB):                         0.00
Kurtosis:                       5.084   Cond. No.                         8.83
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [81]:
# Ordered Logit - n_doses ~ features + dummies

model_n = '3a'
model_name = f"model{model_n}"

# Create the Logit model with a different name
model = OrderedModel(y3,X3,distr='logit').fit(method='bfgs', maxiter=2000)

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

Optimization terminated successfully.
         Current function value: 0.771476
         Iterations: 54
         Function evaluations: 55
         Gradient evaluations: 55


In [82]:
model3a.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:                n_doses   Log-Likelihood:            -2.9483e+05
Model:                   OrderedModel   AIC:                         5.897e+05
Method:            Maximum Likelihood   BIC:                         5.898e+05
Date:                Sun, 26 Mar 2023                                         
Time:                        14:20:41                                         
No. Observations:              382169                                         
Df Residuals:                  382157                                         
Df Model:                           9                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Age                   0.5153      0.005     99.310      0.000       0.505       0.525
Vote diff. PSL-PT    -0.0694      0.004    -19.535      0.000      -0.076      -0.062
Avg. Income           0.1327      0.003     45.131      0.000       0.127       0.138
Female                0.0867      0.008     10.715      0.000       0.071       0.103
Black                -0.0551      0.017     -3.156      0.002      -0.089      -0.021
Brown                -0.1043      0.010    -10.924      0.000      -0.123      -0.086
White                 0.1389      0.008     17.388      0.000       0.123       0.155
Dose1 Pfizer         -1.2852      0.016    -80.625      0.000      -1.316      -1.254
Dose1 SINOVAC         1.0443      0.007    142.846      0.000       1.030       1.059
1/2                  -2.9599      0.012   -245.315      0.000      -2.984      -2.936
2/3                   1.1667      0.003    410.912      0.000       1.161       1.172
3/4                   1.7132      0.003    496.506      0.000       1.706       1.720
=====================================================================================
"""

In [83]:
model3a.prsquared

0.08512053566760203

In [84]:
table = Stargazer([model3a, model1a, model1a2, model2a, model2a2])

In [85]:
print(table.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) \\
\hline \\[-1.8ex]
 1/2 & -2.960$^{***}$ & & & & \\
  & (0.012) & & & & \\
 2/3 & 1.167$^{***}$ & & & & \\
  & (0.003) & & & & \\
 3/4 & 1.713$^{***}$ & & & & \\
  & (0.003) & & & & \\
 Age & 0.515$^{***}$ & 0.472$^{***}$ & 1.175$^{***}$ & -1.311$^{***}$ & -5.716$^{***}$ \\
  & (0.005) & (0.077) & (0.099) & (0.100) & (0.126) \\
 Avg. Income & 0.133$^{***}$ & -0.368$^{***}$ & 0.404$^{***}$ & -1.259$^{***}$ & -2.326$^{***}$ \\
  & (0.003) & (0.044) & (0.057) & (0.057) & (0.074) \\
 Black & -0.055$^{***}$ & 2.949$^{***}$ & 0.409$^{}$ & 1.146$^{***}$ & 3.580$^{***}$ \\
  & (0.017) & (0.272) & (0.354) & (0.369) & (0.473) \\
 Brown & -0.104$^{***}$ & 0.536$^{***}$ & -2.505$^{***}$ & 0.036$^{}$ & 3.533$^{***}$ \\
  & (0.010) & (0.150) & (0.194) & (0.207) & (0.265) \\
 Dose1 Pfizer & -1.285$^{***}$ & -2.515$^{***}$ & & -29.083$^{***}$ & \\
  

--------------------------------------------------------------------
Security Forces

In [86]:
# Defining population

group = 'security_forces'
vaccine = 'dose1_JANSSEN'

df1 = df[df[target[0]] > 0]
df2 = df[df[target[1]] > 0]
df1 = df1.dropna(how='any').copy()
df2 = df2.dropna(how='any').copy()

df3 = df[(df[vaccine] == 0) & (df[group] == 1)]
df1 = df1[(df1[vaccine] == 0) & (df1[group] == 1)]
df2 = df2[(df2[vaccine] == 0) & (df2[group] == 1)]

# setting DF for OLS 1 & 2
X1 = sm.add_constant(df1[list(features) + list(dummies)])
X2 = sm.add_constant(df2[list(features) + list(dummies)])

# setting DF for logit (X3)
X3 = df3[list(features)+ list(dummies)]

# target1 (y1) as diff_dose1_dose2
y1 = df1[target[0]].values.reshape(-1, 1)

# target2 (y2) as diff_dose2_dose3
y2 = df2[target[1]].values.reshape(-1, 1)

# and target3 (y3) as n_doses
y3 = df3[target[2]]

In [87]:
# OLS 1 - diff_dose1_dose2 ~ features + dummies

model_n = '1a'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y1, X1).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.061
Model:                            OLS   Adj. R-squared:                  0.061
Method:                 Least Squares   F-statistic:                     653.7
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:20:46   Log-Likelihood:            -5.3376e+05
No. Observations:               90212   AIC:                         1.068e+06
Df Residuals:                   90202   BIC:                         1.068e+06
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                85.0239      0.492    172.806      0.000      84.060      85.988
Age                  -5.8614      0.552    -10.628      0.000      -6.942      -4.780
Vote diff. PSL-PT     0.9823      0.291      3.376      0.001       0.412       1.553
Avg. Income          -0.2388      0.337     -0.709      0.478      -0.899       0.422
Female               -0.0003      0.828     -0.000      1.000      -1.623       1.622
Black                -0.2497      1.676     -0.149      0.882      -3.536       3.036
Brown                -0.6222      0.771     -0.807      0.420      -2.133       0.889
White                -2.3767      0.760     -3.126      0.002      -3.867      -0.886
Dose1 Pfizer          9.9635      1.210      8.235      0.000       7.592      12.335
Dose1 SINOVAC       -52.2300      0.764    -68.389      0.000     -53.727     -50.733
==============================================================================
Omnibus:                   368994.102   Durbin-Watson:                   1.983
Prob(Omnibus):                  0.000   Jarque-Bera (JB):     692219585334.226
Skew:                         110.175   Prob(JB):                         0.00
Kurtosis:                   13571.697   Cond. No.                         6.36
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [88]:
# OLS 2 - diff_dose2_dose3 ~ features + dummies

model_n = '2a'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y2, X2).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.421
Model:                            OLS   Adj. R-squared:                  0.421
Method:                 Least Squares   F-statistic:                     1366.
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:20:46   Log-Likelihood:                -81277.
No. Observations:               16929   AIC:                         1.626e+05
Df Residuals:                   16919   BIC:                         1.627e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               146.3609      0.399    366.883      0.000     145.579     147.143
Age                   4.4271      0.457      9.686      0.000       3.531       5.323
Vote diff. PSL-PT    -1.3691      0.230     -5.962      0.000      -1.819      -0.919
Avg. Income          -0.9125      0.236     -3.873      0.000      -1.374      -0.451
Female                0.3981      0.570      0.698      0.485      -0.719       1.516
Black                 4.3474      1.246      3.490      0.000       1.905       6.789
Brown                 1.7094      0.629      2.719      0.007       0.477       2.942
White                 5.5703      0.539     10.330      0.000       4.513       6.627
Dose1 Pfizer        -14.0039      1.403     -9.984      0.000     -16.753     -11.255
Dose1 SINOVAC        49.4833      0.480    103.007      0.000      48.542      50.425
==============================================================================
Omnibus:                     5888.271   Durbin-Watson:                   1.707
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            40248.145
Skew:                          -1.503   Prob(JB):                         0.00
Kurtosis:                       9.930   Cond. No.                         7.42
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [89]:
# setting DF for OLS 1 & 2 minuns dummies for vaccines
X1 = sm.add_constant(df1[list(features) + list(dummies2)])
X2 = sm.add_constant(df2[list(features) + list(dummies2)])


In [90]:
# OLS 1 - diff_dose1_dose2 ~ features + dummies minuns dummies for vaccines

model_n = '1a2'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y1, X1).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     127.9
Date:                Sun, 26 Mar 2023   Prob (F-statistic):          5.10e-188
Time:                        14:20:46   Log-Likelihood:            -5.3616e+05
No. Observations:               90212   AIC:                         1.072e+06
Df Residuals:                   90204   BIC:                         1.072e+06
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                72.1751      0.456    158.204      0.000      71.281      73.069
Age                 -16.0638      0.545    -29.487      0.000     -17.131     -14.996
Vote diff. PSL-PT     2.0376      0.297      6.857      0.000       1.455       2.620
Avg. Income           0.0676      0.346      0.196      0.845      -0.610       0.746
Female                0.8948      0.850      1.053      0.292      -0.771       2.561
Black                 1.0054      1.722      0.584      0.559      -2.369       4.380
Brown                -1.4706      0.791     -1.859      0.063      -3.021       0.080
White                -1.9127      0.781     -2.450      0.014      -3.443      -0.382
==============================================================================
Omnibus:                   359821.487   Durbin-Watson:                   1.885
Prob(Omnibus):                  0.000   Jarque-Bera (JB):     558810287481.870
Skew:                         101.646   Prob(JB):                         0.00
Kurtosis:                   12194.167   Cond. No.                         6.24
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [91]:
# OLS 2 - diff_dose2_dose3 ~ features + dummies minuns dummies for vaccines

model_n = '2a2'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y2, X2).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     102.5
Date:                Sun, 26 Mar 2023   Prob (F-statistic):          1.85e-147
Time:                        14:20:46   Log-Likelihood:                -85550.
No. Observations:               16929   AIC:                         1.711e+05
Df Residuals:                   16921   BIC:                         1.712e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               166.3824      0.441    377.141      0.000     165.518     167.247
Age                  14.9745      0.574     26.107      0.000      13.850      16.099
Vote diff. PSL-PT    -1.8879      0.295     -6.389      0.000      -2.467      -1.309
Avg. Income          -1.1321      0.303     -3.735      0.000      -1.726      -0.538
Female               -0.1758      0.734     -0.240      0.811      -1.614       1.263
Black                -1.7094      1.601     -1.068      0.286      -4.848       1.429
Brown                 1.8567      0.809      2.296      0.022       0.271       3.442
White                 1.7903      0.692      2.586      0.010       0.433       3.148
==============================================================================
Omnibus:                     1560.643   Durbin-Watson:                   1.161
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4170.445
Skew:                          -0.526   Prob(JB):                         0.00
Kurtosis:                       5.192   Cond. No.                         6.26
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [92]:
# Ordered Logit - n_doses ~ features + dummies

model_n = '3a'
model_name = f"model{model_n}"

# Create the Logit model with a different name
model = OrderedModel(y3,X3,distr='logit').fit(method='bfgs', maxiter=2000)

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

Optimization terminated successfully.
         Current function value: 0.664157
         Iterations: 53
         Function evaluations: 55
         Gradient evaluations: 55


In [93]:
model3a.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:                n_doses   Log-Likelihood:                -64547.
Model:                   OrderedModel   AIC:                         1.291e+05
Method:            Maximum Likelihood   BIC:                         1.292e+05
Date:                Sun, 26 Mar 2023                                         
Time:                        14:21:12                                         
No. Observations:               97187                                         
Df Residuals:                   97175                                         
Df Model:                           9                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Age                   0.7427      0.014     52.021      0.000       0.715       0.771
Vote diff. PSL-PT    -0.0424      0.007     -5.779      0.000      -0.057      -0.028
Avg. Income           0.1890      0.008     22.849      0.000       0.173       0.205
Female                0.3117      0.021     15.098      0.000       0.271       0.352
Black                 0.1760      0.042      4.156      0.000       0.093       0.259
Brown                -0.0463      0.019     -2.374      0.018      -0.084      -0.008
White                 0.4000      0.019     21.018      0.000       0.363       0.437
Dose1 Pfizer         -0.7504      0.030    -25.054      0.000      -0.809      -0.692
Dose1 SINOVAC         0.8299      0.019     44.627      0.000       0.793       0.866
1/2                  -2.7604      0.017   -163.243      0.000      -2.794      -2.727
2/3                   1.5080      0.004    413.321      0.000       1.501       1.515
3/4                   1.6585      0.017     96.280      0.000       1.625       1.692
=====================================================================================
"""

In [94]:
model3a.prsquared

0.0663720134367578

In [95]:
table = Stargazer([model3a, model1a, model1a2, model2a, model2a2])

In [96]:
print(table.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) \\
\hline \\[-1.8ex]
 1/2 & -2.760$^{***}$ & & & & \\
  & (0.017) & & & & \\
 2/3 & 1.508$^{***}$ & & & & \\
  & (0.004) & & & & \\
 3/4 & 1.659$^{***}$ & & & & \\
  & (0.017) & & & & \\
 Age & 0.743$^{***}$ & -5.861$^{***}$ & -16.064$^{***}$ & 4.427$^{***}$ & 14.974$^{***}$ \\
  & (0.014) & (0.552) & (0.545) & (0.457) & (0.574) \\
 Avg. Income & 0.189$^{***}$ & -0.239$^{}$ & 0.068$^{}$ & -0.913$^{***}$ & -1.132$^{***}$ \\
  & (0.008) & (0.337) & (0.346) & (0.236) & (0.303) \\
 Black & 0.176$^{***}$ & -0.250$^{}$ & 1.005$^{}$ & 4.347$^{***}$ & -1.709$^{}$ \\
  & (0.042) & (1.676) & (1.722) & (1.246) & (1.601) \\
 Brown & -0.046$^{**}$ & -0.622$^{}$ & -1.471$^{*}$ & 1.709$^{***}$ & 1.857$^{**}$ \\
  & (0.019) & (0.771) & (0.791) & (0.629) & (0.809) \\
 Dose1 Pfizer & -0.750$^{***}$ & 9.964$^{***}$ & & -14.004$^{***}$ & \\
  & (0.030) & (

-------------------------------------------------------
Population aged 65-80 yo

In [97]:
# Defining population

group = 'age_65_80'
vaccine = 'dose1_JANSSEN'

df1 = df[df[target[0]] > 0]
df2 = df[df[target[1]] > 0]
df1 = df1.dropna(how='any').copy()
df2 = df2.dropna(how='any').copy()

df3 = df[(df[vaccine] == 0) & (df[group] == 1)]
df1 = df1[(df1[vaccine] == 0) & (df1[group] == 1)]
df2 = df2[(df2[vaccine] == 0) & (df2[group] == 1)]

# setting DF for OLS 1 & 2
X1 = sm.add_constant(df1[list(features) + list(dummies)])
X2 = sm.add_constant(df2[list(features) + list(dummies)])

# setting DF for logit (X3)
X3 = df3[list(features)+ list(dummies)]

# target1 (y1) as diff_dose1_dose2
y1 = df1[target[0]].values.reshape(-1, 1)

# target2 (y2) as diff_dose2_dose3
y2 = df2[target[1]].values.reshape(-1, 1)

# and target3 (y3) as n_doses
y3 = df3[target[2]]

In [98]:
# OLS 1 - diff_dose1_dose2 ~ features + dummies

model_n = '1a'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y1, X1).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.246
Model:                            OLS   Adj. R-squared:                  0.246
Method:                 Least Squares   F-statistic:                 5.320e+04
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:21:18   Log-Likelihood:            -7.3811e+06
No. Observations:             1465049   AIC:                         1.476e+07
Df Residuals:                 1465039   BIC:                         1.476e+07
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                97.2131      0.198    491.226      0.000      96.825      97.601
Age                  -5.0560      0.119    -42.377      0.000      -5.290      -4.822
Vote diff. PSL-PT     0.5631      0.032     17.545      0.000       0.500       0.626
Avg. Income          -1.6033      0.029    -55.994      0.000      -1.659      -1.547
Female                0.2817      0.063      4.478      0.000       0.158       0.405
Black                 1.4376      0.145      9.900      0.000       1.153       1.722
Brown                 0.2872      0.084      3.407      0.001       0.122       0.452
White                -0.5231      0.074     -7.093      0.000      -0.668      -0.379
Dose1 Pfizer        -13.4069      0.392    -34.182      0.000     -14.176     -12.638
Dose1 SINOVAC       -53.7434      0.080   -670.020      0.000     -53.901     -53.586
===============================================================================
Omnibus:                  7000636.655   Durbin-Watson:                    1.132
Prob(Omnibus):                  0.000   Jarque-Bera (JB):   815236673326984.125
Skew:                         182.959   Prob(JB):                          0.00
Kurtosis:                  115566.110   Cond. No.                          27.5
===============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [99]:
# OLS 2 - diff_dose2_dose3 ~ features + dummies

model_n = '2a'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y2, X2).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.219
Model:                            OLS   Adj. R-squared:                  0.219
Method:                 Least Squares   F-statistic:                 3.210e+04
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:21:19   Log-Likelihood:            -4.6552e+06
No. Observations:             1028787   AIC:                         9.310e+06
Df Residuals:                 1028777   BIC:                         9.311e+06
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               136.6229      0.163    838.639      0.000     136.304     136.942
Age                  11.9409      0.094    126.580      0.000      11.756      12.126
Vote diff. PSL-PT    -7.1915      0.023   -306.741      0.000      -7.237      -7.146
Avg. Income          -0.8934      0.020    -44.928      0.000      -0.932      -0.854
Female                0.3136      0.045      6.968      0.000       0.225       0.402
Black                -2.7417      0.104    -26.317      0.000      -2.946      -2.538
Brown                -1.0147      0.062    -16.397      0.000      -1.136      -0.893
White                -1.2955      0.052    -24.744      0.000      -1.398      -1.193
Dose1 Pfizer        -44.0237      0.719    -61.230      0.000     -45.433     -42.614
Dose1 SINOVAC        28.9679      0.065    445.624      0.000      28.840      29.095
==============================================================================
Omnibus:                   432623.979   Durbin-Watson:                   1.573
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          8511120.737
Skew:                          -1.542   Prob(JB):                         0.00
Kurtosis:                      16.749   Cond. No.                         71.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [100]:
# setting DF for OLS 1 & 2 minuns dummies for vaccines
X1 = sm.add_constant(df1[list(features) + list(dummies2)])
X2 = sm.add_constant(df2[list(features) + list(dummies2)])


In [101]:
# OLS 1 - diff_dose1_dose2 ~ features + dummies minuns dummies for vaccines

model_n = '1a2'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y1, X1).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     2656.
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:21:20   Log-Likelihood:            -7.5791e+06
No. Observations:             1465049   AIC:                         1.516e+07
Df Residuals:                 1465041   BIC:                         1.516e+07
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                60.1639      0.214    281.152      0.000      59.744      60.583
Age                  -8.5773      0.135    -63.497      0.000      -8.842      -8.313
Vote diff. PSL-PT    -2.8537      0.036    -78.678      0.000      -2.925      -2.783
Avg. Income          -1.8969      0.033    -57.882      0.000      -1.961      -1.833
Female               -0.1916      0.072     -2.661      0.008      -0.333      -0.050
Black                -1.4238      0.166     -8.569      0.000      -1.749      -1.098
Brown                 2.6593      0.096     27.589      0.000       2.470       2.848
White                -2.7432      0.084    -32.530      0.000      -2.908      -2.578
===============================================================================
Omnibus:                  6220332.928   Durbin-Watson:                    0.817
Prob(Omnibus):                  0.000   Jarque-Bera (JB):   279411760474212.125
Skew:                         122.451   Prob(JB):                          0.00
Kurtosis:                   67657.866   Cond. No.                          14.1
===============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [102]:
# OLS 2 - diff_dose2_dose3 ~ features + dummies minuns dummies for vaccines

model_n = '2a2'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y2, X2).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.061
Model:                            OLS   Adj. R-squared:                  0.061
Method:                 Least Squares   F-statistic:                     9602.
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:21:20   Log-Likelihood:            -4.7500e+06
No. Observations:             1028787   AIC:                         9.500e+06
Df Residuals:                 1028779   BIC:                         9.500e+06
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               164.6276      0.164   1002.362      0.000     164.306     164.950
Age                   9.6941      0.103     93.928      0.000       9.492       9.896
Vote diff. PSL-PT    -5.9229      0.026   -232.192      0.000      -5.973      -5.873
Avg. Income          -0.8796      0.022    -40.344      0.000      -0.922      -0.837
Female                0.5324      0.049     10.788      0.000       0.436       0.629
Black                -1.8146      0.114    -15.888      0.000      -2.038      -1.591
Brown                -2.3248      0.068    -34.299      0.000      -2.458      -2.192
White                -0.5159      0.057     -8.992      0.000      -0.628      -0.403
==============================================================================
Omnibus:                   393895.601   Durbin-Watson:                   1.300
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          4460917.991
Skew:                          -1.523   Prob(JB):                         0.00
Kurtosis:                      12.736   Cond. No.                         16.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [103]:
# Ordered Logit - n_doses ~ features + dummies

model_n = '3a'
model_name = f"model{model_n}"

# Create the Logit model with a different name
model = OrderedModel(y3,X3,distr='logit').fit(method='bfgs', maxiter=2000)

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

Optimization terminated successfully.
         Current function value: 0.742515
         Iterations: 59
         Function evaluations: 60
         Gradient evaluations: 60


In [104]:
model3a.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:                n_doses   Log-Likelihood:            -1.1442e+06
Model:                   OrderedModel   AIC:                         2.288e+06
Method:            Maximum Likelihood   BIC:                         2.289e+06
Date:                Sun, 26 Mar 2023                                         
Time:                        14:27:02                                         
No. Observations:             1540956                                         
Df Residuals:                 1540944                                         
Df Model:                           9                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Age                   0.3487      0.006     54.304      0.000       0.336       0.361
Vote diff. PSL-PT     0.1709      0.002     95.190      0.000       0.167       0.174
Avg. Income           0.2295      0.002    128.772      0.000       0.226       0.233
Female                0.0484      0.004     13.601      0.000       0.041       0.055
Black                 0.0895      0.008     11.023      0.000       0.074       0.105
Brown                 0.0840      0.005     18.003      0.000       0.075       0.093
White                 0.2684      0.004     63.387      0.000       0.260       0.277
Dose1 Pfizer         -3.1188      0.014   -224.166      0.000      -3.146      -3.092
Dose1 SINOVAC         0.9439      0.004    224.186      0.000       0.936       0.952
1/2                  -1.9866      0.011   -179.663      0.000      -2.008      -1.965
2/3                   0.9799      0.002    602.071      0.000       0.977       0.983
3/4                   1.7759      0.002   1053.940      0.000       1.773       1.779
=====================================================================================
"""

In [105]:
model3a.prsquared

0.07527147691687675

In [106]:
table = Stargazer([model3a, model1a, model1a2, model2a, model2a2])

In [107]:
print(table.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) \\
\hline \\[-1.8ex]
 1/2 & -1.987$^{***}$ & & & & \\
  & (0.011) & & & & \\
 2/3 & 0.980$^{***}$ & & & & \\
  & (0.002) & & & & \\
 3/4 & 1.776$^{***}$ & & & & \\
  & (0.002) & & & & \\
 Age & 0.349$^{***}$ & -5.056$^{***}$ & -8.577$^{***}$ & 11.941$^{***}$ & 9.694$^{***}$ \\
  & (0.006) & (0.119) & (0.135) & (0.094) & (0.103) \\
 Avg. Income & 0.230$^{***}$ & -1.603$^{***}$ & -1.897$^{***}$ & -0.893$^{***}$ & -0.880$^{***}$ \\
  & (0.002) & (0.029) & (0.033) & (0.020) & (0.022) \\
 Black & 0.090$^{***}$ & 1.438$^{***}$ & -1.424$^{***}$ & -2.742$^{***}$ & -1.815$^{***}$ \\
  & (0.008) & (0.145) & (0.166) & (0.104) & (0.114) \\
 Brown & 0.084$^{***}$ & 0.287$^{***}$ & 2.659$^{***}$ & -1.015$^{***}$ & -2.325$^{***}$ \\
  & (0.005) & (0.084) & (0.096) & (0.062) & (0.068) \\
 Dose1 Pfizer & -3.119$^{***}$ & -13.407$^{***}$ & & -44.024$^{**

--------------------------------
**Regions**

In [108]:
df['nome_regiao'].unique()

array(['Sul', 'Sudeste', 'Centro-Oeste', 'Nordeste', 'Norte'],
      dtype=object)

In [109]:
# Defining population

group = 'nome_regiao'
region = 'Norte'
vaccine = 'dose1_JANSSEN'

df1 = df[df[target[0]] > 0]
df2 = df[df[target[1]] > 0]
df1 = df1.dropna(how='any').copy()
df2 = df2.dropna(how='any').copy()

df3 = df[(df[vaccine] == 0) & (df[group] == region)]
df1 = df1[(df1[vaccine] == 0) & (df1[group] == region)]
df2 = df2[(df2[vaccine] == 0) & (df2[group] == region)]

# setting DF for OLS 1 & 2
X1 = sm.add_constant(df1[list(features) + list(dummies)])
X2 = sm.add_constant(df2[list(features) + list(dummies)])

# setting DF for logit (X3)
X3 = df3[list(features)+ list(dummies)]

# target1 (y1) as diff_dose1_dose2
y1 = df1[target[0]].values.reshape(-1, 1)

# target2 (y2) as diff_dose2_dose3
y2 = df2[target[1]].values.reshape(-1, 1)

# and target3 (y3) as n_doses
y3 = df3[target[2]]

In [110]:
# OLS 1 - diff_dose1_dose2 ~ features + dummies

model_n = '1a'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y1, X1).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.451
Model:                            OLS   Adj. R-squared:                  0.451
Method:                 Least Squares   F-statistic:                 1.094e+05
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:27:10   Log-Likelihood:            -5.4285e+06
No. Observations:             1196017   AIC:                         1.086e+07
Df Residuals:                 1196007   BIC:                         1.086e+07
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                82.0029      0.073   1121.742      0.000      81.860      82.146
Age                   0.8749      0.026     33.777      0.000       0.824       0.926
Vote diff. PSL-PT    -3.2259      0.067    -48.237      0.000      -3.357      -3.095
Avg. Income           1.0512      0.051     20.513      0.000       0.951       1.152
Female               -0.2369      0.042     -5.670      0.000      -0.319      -0.155
Black                 0.5262      0.193      2.726      0.006       0.148       0.905
Brown                 0.4091      0.045      9.164      0.000       0.322       0.497
White                -0.2436      0.077     -3.151      0.002      -0.395      -0.092
Dose1 Pfizer         -1.4108      0.055    -25.605      0.000      -1.519      -1.303
Dose1 SINOVAC       -49.3441      0.053   -939.255      0.000     -49.447     -49.241
==============================================================================
Omnibus:                   761073.554   Durbin-Watson:                   1.911
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         18794169.157
Skew:                           2.665   Prob(JB):                         0.00
Kurtosis:                      21.674   Cond. No.                         16.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [111]:
# OLS 2 - diff_dose2_dose3 ~ features + dummies

model_n = '2a'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y2, X2).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.262
Model:                            OLS   Adj. R-squared:                  0.262
Method:                 Least Squares   F-statistic:                     6287.
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:27:10   Log-Likelihood:            -7.9118e+05
No. Observations:              159578   AIC:                         1.582e+06
Df Residuals:                  159568   BIC:                         1.582e+06
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               167.7824      0.326    515.280      0.000     167.144     168.421
Age                   4.1910      0.113     37.217      0.000       3.970       4.412
Vote diff. PSL-PT     5.4193      0.279     19.425      0.000       4.873       5.966
Avg. Income          -2.8464      0.196    -14.558      0.000      -3.230      -2.463
Female                3.6566      0.176     20.755      0.000       3.311       4.002
Black                -1.7098      0.793     -2.155      0.031      -3.265      -0.155
Brown                 0.0683      0.189      0.361      0.718      -0.302       0.439
White                 1.6987      0.297      5.720      0.000       1.117       2.281
Dose1 Pfizer        -76.4986      0.801    -95.498      0.000     -78.069     -74.929
Dose1 SINOVAC        37.1763      0.199    187.159      0.000      36.787      37.566
==============================================================================
Omnibus:                    51521.344   Durbin-Watson:                   1.895
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           277364.211
Skew:                          -1.461   Prob(JB):                         0.00
Kurtosis:                       8.760   Cond. No.                         17.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [112]:
# setting DF for OLS 1 & 2 minuns dummies for vaccines
X1 = sm.add_constant(df1[list(features) + list(dummies2)])
X2 = sm.add_constant(df2[list(features) + list(dummies2)])


In [113]:
# OLS 1 - diff_dose1_dose2 ~ features + dummies minuns dummies for vaccines

model_n = '1a2'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y1, X1).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     1191.
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:27:11   Log-Likelihood:            -5.7834e+06
No. Observations:             1196017   AIC:                         1.157e+07
Df Residuals:                 1196009   BIC:                         1.157e+07
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                69.6622      0.093    752.840      0.000      69.481      69.844
Age                  -1.4884      0.031    -48.179      0.000      -1.549      -1.428
Vote diff. PSL-PT    -4.1041      0.090    -45.753      0.000      -4.280      -3.928
Avg. Income           0.3195      0.069      4.646      0.000       0.185       0.454
Female               -0.8623      0.056    -15.345      0.000      -0.972      -0.752
Black                -0.2990      0.260     -1.151      0.250      -0.808       0.210
Brown                -0.0319      0.060     -0.532      0.595      -0.150       0.086
White                -1.1212      0.104    -10.783      0.000      -1.325      -0.917
==============================================================================
Omnibus:                   137551.242   Durbin-Watson:                   1.025
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           535212.437
Skew:                           0.538   Prob(JB):                         0.00
Kurtosis:                       6.096   Cond. No.                         16.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [114]:
# OLS 2 - diff_dose2_dose3 ~ features + dummies minuns dummies for vaccines

model_n = '2a2'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y2, X2).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     241.2
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:27:11   Log-Likelihood:            -8.1455e+05
No. Observations:              159578   AIC:                         1.629e+06
Df Residuals:                  159570   BIC:                         1.629e+06
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               191.4341      0.339    565.143      0.000     190.770     192.098
Age                  -3.6036      0.115    -31.437      0.000      -3.828      -3.379
Vote diff. PSL-PT     2.6755      0.322      8.298      0.000       2.044       3.307
Avg. Income          -0.4730      0.226     -2.094      0.036      -0.916      -0.030
Female                4.1960      0.204     20.574      0.000       3.796       4.596
Black                -0.5199      0.919     -0.566      0.571      -2.320       1.280
Brown                 0.7089      0.219      3.240      0.001       0.280       1.138
White                 2.1568      0.344      6.273      0.000       1.483       2.831
==============================================================================
Omnibus:                    46095.433   Durbin-Watson:                   1.454
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           179951.348
Skew:                          -1.400   Prob(JB):                         0.00
Kurtosis:                       7.384   Cond. No.                         16.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [115]:
# Ordered Logit - n_doses ~ features + dummies

model_n = '3a'
model_name = f"model{model_n}"

# Create the Logit model with a different name
model = OrderedModel(y3,X3,distr='logit').fit(method='bfgs', maxiter=2000)

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

Optimization terminated successfully.
         Current function value: 0.732774
         Iterations: 64
         Function evaluations: 65
         Gradient evaluations: 65


In [116]:
model3a.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:                n_doses   Log-Likelihood:            -1.1420e+06
Model:                   OrderedModel   AIC:                         2.284e+06
Method:            Maximum Likelihood   BIC:                         2.284e+06
Date:                Sun, 26 Mar 2023                                         
Time:                        14:33:39                                         
No. Observations:             1558449                                         
Df Residuals:                 1558437                                         
Df Model:                           9                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Age                   0.8357      0.002    348.539      0.000       0.831       0.840
Vote diff. PSL-PT     0.2260      0.006     39.570      0.000       0.215       0.237
Avg. Income           0.0635      0.004     14.427      0.000       0.055       0.072
Female                0.1968      0.004     56.173      0.000       0.190       0.204
Black                -0.2108      0.016    -12.934      0.000      -0.243      -0.179
Brown                 0.0149      0.004      3.976      0.000       0.008       0.022
White                 0.2801      0.007     41.650      0.000       0.267       0.293
Dose1 Pfizer         -0.7364      0.004   -163.933      0.000      -0.745      -0.728
Dose1 SINOVAC         0.8616      0.005    172.756      0.000       0.852       0.871
1/2                  -1.9388      0.007   -294.696      0.000      -1.952      -1.926
2/3                   1.4280      0.001   1448.170      0.000       1.426       1.430
3/4                   1.5400      0.005    330.401      0.000       1.531       1.549
=====================================================================================
"""

In [117]:
model3a.prsquared

0.13927728256312255

In [118]:
table = Stargazer([model3a, model1a, model1a2, model2a, model2a2])

In [119]:
print(table.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) \\
\hline \\[-1.8ex]
 1/2 & -1.939$^{***}$ & & & & \\
  & (0.007) & & & & \\
 2/3 & 1.428$^{***}$ & & & & \\
  & (0.001) & & & & \\
 3/4 & 1.540$^{***}$ & & & & \\
  & (0.005) & & & & \\
 Age & 0.836$^{***}$ & 0.875$^{***}$ & -1.488$^{***}$ & 4.191$^{***}$ & -3.604$^{***}$ \\
  & (0.002) & (0.026) & (0.031) & (0.113) & (0.115) \\
 Avg. Income & 0.063$^{***}$ & 1.051$^{***}$ & 0.320$^{***}$ & -2.846$^{***}$ & -0.473$^{**}$ \\
  & (0.004) & (0.051) & (0.069) & (0.196) & (0.226) \\
 Black & -0.211$^{***}$ & 0.526$^{***}$ & -0.299$^{}$ & -1.710$^{**}$ & -0.520$^{}$ \\
  & (0.016) & (0.193) & (0.260) & (0.793) & (0.919) \\
 Brown & 0.015$^{***}$ & 0.409$^{***}$ & -0.032$^{}$ & 0.068$^{}$ & 0.709$^{***}$ \\
  & (0.004) & (0.045) & (0.060) & (0.189) & (0.219) \\
 Dose1 Pfizer & -0.736$^{***}$ & -1.411$^{***}$ & & -76.499$^{***}$ & \\
  & (0.00

In [120]:
# Defining population

group = 'nome_regiao'
region = 'Nordeste'
vaccine = 'dose1_JANSSEN'

df1 = df[df[target[0]] > 0]
df2 = df[df[target[1]] > 0]
df1 = df1.dropna(how='any').copy()
df2 = df2.dropna(how='any').copy()

df3 = df[(df[vaccine] == 0) & (df[group] == region)]
df1 = df1[(df1[vaccine] == 0) & (df1[group] == region)]
df2 = df2[(df2[vaccine] == 0) & (df2[group] == region)]

# setting DF for OLS 1 & 2
X1 = sm.add_constant(df1[list(features) + list(dummies)])
X2 = sm.add_constant(df2[list(features) + list(dummies)])

# setting DF for logit (X3)
X3 = df3[list(features)+ list(dummies)]

# target1 (y1) as diff_dose1_dose2
y1 = df1[target[0]].values.reshape(-1, 1)

# target2 (y2) as diff_dose2_dose3
y2 = df2[target[1]].values.reshape(-1, 1)

# and target3 (y3) as n_doses
y3 = df3[target[2]]

In [121]:
# OLS 1 - diff_dose1_dose2 ~ features + dummies

model_n = '1a'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y1, X1).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.219
Model:                            OLS   Adj. R-squared:                  0.219
Method:                 Least Squares   F-statistic:                 4.621e+04
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:33:47   Log-Likelihood:            -7.7371e+06
No. Observations:             1485483   AIC:                         1.547e+07
Df Residuals:                 1485473   BIC:                         1.547e+07
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                87.3448      0.095    921.884      0.000      87.159      87.531
Age                   2.0779      0.042     49.929      0.000       1.996       2.159
Vote diff. PSL-PT     1.3218      0.054     24.697      0.000       1.217       1.427
Avg. Income          -1.4382      0.038    -38.033      0.000      -1.512      -1.364
Female                0.2286      0.074      3.095      0.002       0.084       0.373
Black                 3.8092      0.190     20.002      0.000       3.436       4.182
Brown                 1.5511      0.085     18.348      0.000       1.385       1.717
White                 0.8764      0.117      7.486      0.000       0.647       1.106
Dose1 Pfizer         -3.6556      0.099    -37.025      0.000      -3.849      -3.462
Dose1 SINOVAC       -51.6070      0.085   -607.303      0.000     -51.774     -51.440
===============================================================================
Omnibus:                  7872908.986   Durbin-Watson:                    1.925
Prob(Omnibus):                  0.000   Jarque-Bera (JB):   793447401878719.250
Skew:                         265.700   Prob(JB):                          0.00
Kurtosis:                  113223.760   Cond. No.                          8.98
===============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [122]:
# OLS 2 - diff_dose2_dose3 ~ features + dummies

model_n = '2a'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y2, X2).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.386
Model:                            OLS   Adj. R-squared:                  0.386
Method:                 Least Squares   F-statistic:                 1.869e+04
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:33:47   Log-Likelihood:            -1.3102e+06
No. Observations:              267519   AIC:                         2.621e+06
Df Residuals:                  267509   BIC:                         2.621e+06
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               145.8519      0.195    747.271      0.000     145.469     146.234
Age                   0.2965      0.072      4.135      0.000       0.156       0.437
Vote diff. PSL-PT     1.0466      0.097     10.836      0.000       0.857       1.236
Avg. Income          -0.4118      0.057     -7.181      0.000      -0.524      -0.299
Female                4.5012      0.131     34.394      0.000       4.245       4.758
Black                 0.6181      0.364      1.698      0.089      -0.095       1.331
Brown                -0.7323      0.152     -4.829      0.000      -1.030      -0.435
White                 0.7886      0.190      4.156      0.000       0.417       1.161
Dose1 Pfizer        -46.6805      0.450   -103.761      0.000     -47.562     -45.799
Dose1 SINOVAC        50.1125      0.140    356.958      0.000      49.837      50.388
==============================================================================
Omnibus:                    78097.820   Durbin-Watson:                   1.837
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           542441.808
Skew:                          -1.227   Prob(JB):                         0.00
Kurtosis:                       9.530   Cond. No.                         14.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [123]:
# setting DF for OLS 1 & 2 minuns dummies for vaccines
X1 = sm.add_constant(df1[list(features) + list(dummies2)])
X2 = sm.add_constant(df2[list(features) + list(dummies2)])


In [124]:
# OLS 1 - diff_dose1_dose2 ~ features + dummies minuns dummies for vaccines

model_n = '1a2'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y1, X1).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     697.7
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:33:48   Log-Likelihood:            -7.9180e+06
No. Observations:             1485483   AIC:                         1.584e+07
Df Residuals:                 1485475   BIC:                         1.584e+07
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                70.9082      0.094    751.156      0.000      70.723      71.093
Age                  -2.1081      0.042    -49.612      0.000      -2.191      -2.025
Vote diff. PSL-PT     1.4933      0.060     24.704      0.000       1.375       1.612
Avg. Income          -1.5474      0.043    -36.232      0.000      -1.631      -1.464
Female               -0.1931      0.083     -2.314      0.021      -0.357      -0.030
Black                 2.5070      0.215     11.656      0.000       2.085       2.929
Brown                 1.0375      0.095     10.867      0.000       0.850       1.225
White                 0.1400      0.132      1.059      0.290      -0.119       0.399
===============================================================================
Omnibus:                  7104625.766   Durbin-Watson:                    1.495
Prob(Omnibus):                  0.000   Jarque-Bera (JB):   299961819760070.750
Skew:                         184.269   Prob(JB):                          0.00
Kurtosis:                   69617.335   Cond. No.                          8.73
===============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [125]:
# OLS 2 - diff_dose2_dose3 ~ features + dummies minuns dummies for vaccines

model_n = '2a2'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y2, X2).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     702.8
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:33:48   Log-Likelihood:            -1.3731e+06
No. Observations:              267519   AIC:                         2.746e+06
Df Residuals:                  267511   BIC:                         2.746e+06
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               175.3376      0.214    819.159      0.000     174.918     175.757
Age                   4.5671      0.089     51.344      0.000       4.393       4.741
Vote diff. PSL-PT     2.6569      0.122     21.771      0.000       2.418       2.896
Avg. Income          -0.9622      0.072    -13.274      0.000      -1.104      -0.820
Female                6.4212      0.165     38.824      0.000       6.097       6.745
Black                 0.5705      0.460      1.239      0.215      -0.332       1.473
Brown                -0.1257      0.192     -0.656      0.512      -0.502       0.250
White                 1.5061      0.240      6.276      0.000       1.036       1.976
==============================================================================
Omnibus:                    37692.123   Durbin-Watson:                   1.179
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            87730.006
Skew:                          -0.826   Prob(JB):                         0.00
Kurtosis:                       5.268   Cond. No.                         11.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [126]:
# Ordered Logit - n_doses ~ features + dummies

model_n = '3a'
model_name = f"model{model_n}"

# Create the Logit model with a different name
model = OrderedModel(y3,X3,distr='logit').fit(method='bfgs', maxiter=2000)

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

Optimization terminated successfully.
         Current function value: 0.777616
         Iterations: 49
         Function evaluations: 50
         Gradient evaluations: 50


In [127]:
model3a.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:                n_doses   Log-Likelihood:            -1.5117e+06
Model:                   OrderedModel   AIC:                         3.023e+06
Method:            Maximum Likelihood   BIC:                         3.024e+06
Date:                Sun, 26 Mar 2023                                         
Time:                        14:40:04                                         
No. Observations:             1944042                                         
Df Residuals:                 1944030                                         
Df Model:                           9                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Age                   0.8393      0.002    442.340      0.000       0.836       0.843
Vote diff. PSL-PT     0.1860      0.002     84.512      0.000       0.182       0.190
Avg. Income           0.2168      0.002    130.771      0.000       0.214       0.220
Female                0.1657      0.003     53.775      0.000       0.160       0.172
Black                -0.2823      0.008    -36.268      0.000      -0.298      -0.267
Brown                -0.1343      0.004    -38.278      0.000      -0.141      -0.127
White                 0.0711      0.005     14.275      0.000       0.061       0.081
Dose1 Pfizer         -0.5155      0.004   -126.964      0.000      -0.524      -0.508
Dose1 SINOVAC         0.9138      0.004    233.880      0.000       0.906       0.921
1/2                  -1.6503      0.004   -384.340      0.000      -1.659      -1.642
2/3                   1.3453      0.001   1526.521      0.000       1.344       1.347
3/4                   1.4791      0.003    483.260      0.000       1.473       1.485
=====================================================================================
"""

In [128]:
model3a.prsquared

0.15657098189526908

In [129]:
table = Stargazer([model3a, model1a, model1a2, model2a, model2a2])

In [130]:
print(table.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) \\
\hline \\[-1.8ex]
 1/2 & -1.650$^{***}$ & & & & \\
  & (0.004) & & & & \\
 2/3 & 1.345$^{***}$ & & & & \\
  & (0.001) & & & & \\
 3/4 & 1.479$^{***}$ & & & & \\
  & (0.003) & & & & \\
 Age & 0.839$^{***}$ & 2.078$^{***}$ & -2.108$^{***}$ & 0.296$^{***}$ & 4.567$^{***}$ \\
  & (0.002) & (0.042) & (0.042) & (0.072) & (0.089) \\
 Avg. Income & 0.217$^{***}$ & -1.438$^{***}$ & -1.547$^{***}$ & -0.412$^{***}$ & -0.962$^{***}$ \\
  & (0.002) & (0.038) & (0.043) & (0.057) & (0.072) \\
 Black & -0.282$^{***}$ & 3.809$^{***}$ & 2.507$^{***}$ & 0.618$^{*}$ & 0.570$^{}$ \\
  & (0.008) & (0.190) & (0.215) & (0.364) & (0.460) \\
 Brown & -0.134$^{***}$ & 1.551$^{***}$ & 1.037$^{***}$ & -0.732$^{***}$ & -0.126$^{}$ \\
  & (0.004) & (0.085) & (0.095) & (0.152) & (0.192) \\
 Dose1 Pfizer & -0.516$^{***}$ & -3.656$^{***}$ & & -46.680$^{***}$ & \\
  &

In [131]:
# Defining population

group = 'nome_regiao'
region = 'Centro-Oeste'
vaccine = 'dose1_JANSSEN'

df1 = df[df[target[0]] > 0]
df2 = df[df[target[1]] > 0]
df1 = df1.dropna(how='any').copy()
df2 = df2.dropna(how='any').copy()

df3 = df[(df[vaccine] == 0) & (df[group] == region)]
df1 = df1[(df1[vaccine] == 0) & (df1[group] == region)]
df2 = df2[(df2[vaccine] == 0) & (df2[group] == region)]

# setting DF for OLS 1 & 2
X1 = sm.add_constant(df1[list(features) + list(dummies)])
X2 = sm.add_constant(df2[list(features) + list(dummies)])

# setting DF for logit (X3)
X3 = df3[list(features)+ list(dummies)]

# target1 (y1) as diff_dose1_dose2
y1 = df1[target[0]].values.reshape(-1, 1)

# target2 (y2) as diff_dose2_dose3
y2 = df2[target[1]].values.reshape(-1, 1)

# and target3 (y3) as n_doses
y3 = df3[target[2]]

In [132]:
# OLS 1 - diff_dose1_dose2 ~ features + dummies

model_n = '1a'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y1, X1).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.463
Model:                            OLS   Adj. R-squared:                  0.463
Method:                 Least Squares   F-statistic:                 7.417e+04
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:40:12   Log-Likelihood:            -3.4961e+06
No. Observations:              774800   AIC:                         6.992e+06
Df Residuals:                  774790   BIC:                         6.992e+06
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                83.5657      0.062   1351.785      0.000      83.445      83.687
Age                   2.1432      0.029     73.870      0.000       2.086       2.200
Vote diff. PSL-PT    -9.4150      0.064   -147.449      0.000      -9.540      -9.290
Avg. Income           0.8172      0.033     24.709      0.000       0.752       0.882
Female               -0.7322      0.051    -14.496      0.000      -0.831      -0.633
Black                 0.3515      0.125      2.813      0.005       0.107       0.596
Brown                -0.1129      0.065     -1.737      0.082      -0.240       0.015
White                -1.9958      0.063    -31.519      0.000      -2.120      -1.872
Dose1 Pfizer         -8.1116      0.064   -127.125      0.000      -8.237      -7.987
Dose1 SINOVAC       -46.9621      0.062   -758.960      0.000     -47.083     -46.841
==============================================================================
Omnibus:                   584645.580   Durbin-Watson:                   1.678
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         18645403.434
Skew:                           3.340   Prob(JB):                         0.00
Kurtosis:                      26.085   Cond. No.                         7.00
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [133]:
# OLS 2 - diff_dose2_dose3 ~ features + dummies

model_n = '2a'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y2, X2).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.263
Model:                            OLS   Adj. R-squared:                  0.263
Method:                 Least Squares   F-statistic:                     8256.
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:40:12   Log-Likelihood:            -1.0356e+06
No. Observations:              208399   AIC:                         2.071e+06
Df Residuals:                  208389   BIC:                         2.071e+06
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               147.3206      0.205    719.162      0.000     146.919     147.722
Age                  -2.1447      0.092    -23.382      0.000      -2.324      -1.965
Vote diff. PSL-PT   -16.9508      0.185    -91.772      0.000     -17.313     -16.589
Avg. Income           3.7312      0.091     40.902      0.000       3.552       3.910
Female                4.3055      0.156     27.650      0.000       4.000       4.611
Black                -1.4321      0.400     -3.583      0.000      -2.215      -0.649
Brown                -2.1350      0.217     -9.853      0.000      -2.560      -1.710
White                -4.0221      0.192    -20.900      0.000      -4.399      -3.645
Dose1 Pfizer        -19.2424      0.285    -67.412      0.000     -19.802     -18.683
Dose1 SINOVAC        34.1443      0.172    198.499      0.000      33.807      34.481
==============================================================================
Omnibus:                    11396.657   Durbin-Watson:                   1.772
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            46815.045
Skew:                          -0.048   Prob(JB):                         0.00
Kurtosis:                       5.320   Cond. No.                         9.32
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [134]:
# setting DF for OLS 1 & 2 minuns dummies for vaccines
X1 = sm.add_constant(df1[list(features) + list(dummies2)])
X2 = sm.add_constant(df2[list(features) + list(dummies2)])


In [135]:
# OLS 1 - diff_dose1_dose2 ~ features + dummies minuns dummies for vaccines

model_n = '1a2'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y1, X1).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     4745.
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:40:12   Log-Likelihood:            -3.7206e+06
No. Observations:              774800   AIC:                         7.441e+06
Df Residuals:                  774792   BIC:                         7.441e+06
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                67.9428      0.070    967.304      0.000      67.805      68.080
Age                  -2.3680      0.035    -67.503      0.000      -2.437      -2.299
Vote diff. PSL-PT   -11.3161      0.085   -132.849      0.000     -11.483     -11.149
Avg. Income           1.0371      0.044     23.476      0.000       0.951       1.124
Female               -1.0131      0.067    -15.013      0.000      -1.145      -0.881
Black                -0.3184      0.167     -1.907      0.056      -0.646       0.009
Brown                -0.8619      0.087     -9.923      0.000      -1.032      -0.692
White                -3.2342      0.085    -38.241      0.000      -3.400      -3.068
==============================================================================
Omnibus:                   151471.990   Durbin-Watson:                   0.989
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           774340.020
Skew:                           0.857   Prob(JB):                         0.00
Kurtosis:                       7.588   Cond. No.                         6.60
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [136]:
# OLS 2 - diff_dose2_dose3 ~ features + dummies minuns dummies for vaccines

model_n = '2a2'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y2, X2).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.069
Model:                            OLS   Adj. R-squared:                  0.069
Method:                 Least Squares   F-statistic:                     2220.
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:40:12   Log-Likelihood:            -1.0599e+06
No. Observations:              208399   AIC:                         2.120e+06
Df Residuals:                  208391   BIC:                         2.120e+06
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               156.5787      0.211    743.322      0.000     156.166     156.992
Age                   6.5336      0.094     69.324      0.000       6.349       6.718
Vote diff. PSL-PT   -18.3458      0.207    -88.553      0.000     -18.752     -17.940
Avg. Income           4.0940      0.102     39.965      0.000       3.893       4.295
Female                5.3887      0.175     30.814      0.000       5.046       5.731
Black                -1.4185      0.449     -3.159      0.002      -2.299      -0.538
Brown                -2.2466      0.243     -9.228      0.000      -2.724      -1.769
White                -3.9757      0.216    -18.387      0.000      -4.399      -3.552
==============================================================================
Omnibus:                    18089.979   Durbin-Watson:                   1.464
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            45006.327
Skew:                           0.519   Prob(JB):                         0.00
Kurtosis:                       5.026   Cond. No.                         8.76
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [137]:
# Ordered Logit - n_doses ~ features + dummies

model_n = '3a'
model_name = f"model{model_n}"

# Create the Logit model with a different name
model = OrderedModel(y3,X3,distr='logit').fit(method='bfgs', maxiter=2000)

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

Optimization terminated successfully.
         Current function value: 0.807263
         Iterations: 50
         Function evaluations: 51
         Gradient evaluations: 51


In [138]:
model3a.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:                n_doses   Log-Likelihood:            -7.6048e+05
Model:                   OrderedModel   AIC:                         1.521e+06
Method:            Maximum Likelihood   BIC:                         1.521e+06
Date:                Sun, 26 Mar 2023                                         
Time:                        14:43:43                                         
No. Observations:              942045                                         
Df Residuals:                  942033                                         
Df Model:                           9                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Age                   0.8102      0.003    301.589      0.000       0.805       0.815
Vote diff. PSL-PT     0.8580      0.006    151.358      0.000       0.847       0.869
Avg. Income           0.0139      0.003      4.815      0.000       0.008       0.020
Female                0.2457      0.004     56.844      0.000       0.237       0.254
Black                -0.0658      0.011     -6.180      0.000      -0.087      -0.045
Brown                -0.0151      0.005     -2.759      0.006      -0.026      -0.004
White                 0.2942      0.005     53.817      0.000       0.283       0.305
Dose1 Pfizer         -0.7379      0.006   -130.734      0.000      -0.749      -0.727
Dose1 SINOVAC         0.3920      0.006     70.308      0.000       0.381       0.403
1/2                  -2.0514      0.006   -341.055      0.000      -2.063      -2.040
2/3                   1.2723      0.001   1007.788      0.000       1.270       1.275
3/4                   1.6866      0.004    400.788      0.000       1.678       1.695
=====================================================================================
"""

In [139]:
model3a.prsquared

0.15704173455692272

In [140]:
table = Stargazer([model3a, model1a, model1a2, model2a, model2a2])

In [141]:
print(table.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) \\
\hline \\[-1.8ex]
 1/2 & -2.051$^{***}$ & & & & \\
  & (0.006) & & & & \\
 2/3 & 1.272$^{***}$ & & & & \\
  & (0.001) & & & & \\
 3/4 & 1.687$^{***}$ & & & & \\
  & (0.004) & & & & \\
 Age & 0.810$^{***}$ & 2.143$^{***}$ & -2.368$^{***}$ & -2.145$^{***}$ & 6.534$^{***}$ \\
  & (0.003) & (0.029) & (0.035) & (0.092) & (0.094) \\
 Avg. Income & 0.014$^{***}$ & 0.817$^{***}$ & 1.037$^{***}$ & 3.731$^{***}$ & 4.094$^{***}$ \\
  & (0.003) & (0.033) & (0.044) & (0.091) & (0.102) \\
 Black & -0.066$^{***}$ & 0.351$^{***}$ & -0.318$^{*}$ & -1.432$^{***}$ & -1.418$^{***}$ \\
  & (0.011) & (0.125) & (0.167) & (0.400) & (0.449) \\
 Brown & -0.015$^{***}$ & -0.113$^{*}$ & -0.862$^{***}$ & -2.135$^{***}$ & -2.247$^{***}$ \\
  & (0.005) & (0.065) & (0.087) & (0.217) & (0.243) \\
 Dose1 Pfizer & -0.738$^{***}$ & -8.112$^{***}$ & & -19.242$^{***}$ & 

In [142]:
# Defining population

group = 'nome_regiao'
region = 'Sudeste'
vaccine = 'dose1_JANSSEN'

df1 = df[df[target[0]] > 0]
df2 = df[df[target[1]] > 0]
df1 = df1.dropna(how='any').copy()
df2 = df2.dropna(how='any').copy()

df3 = df[(df[vaccine] == 0) & (df[group] == region)]
df1 = df1[(df1[vaccine] == 0) & (df1[group] == region)]
df2 = df2[(df2[vaccine] == 0) & (df2[group] == region)]

# setting DF for OLS 1 & 2
X1 = sm.add_constant(df1[list(features) + list(dummies)])
X2 = sm.add_constant(df2[list(features) + list(dummies)])

# setting DF for logit (X3)
X3 = df3[list(features)+ list(dummies)]

# target1 (y1) as diff_dose1_dose2
y1 = df1[target[0]].values.reshape(-1, 1)

# target2 (y2) as diff_dose2_dose3
y2 = df2[target[1]].values.reshape(-1, 1)

# and target3 (y3) as n_doses
y3 = df3[target[2]]

In [143]:
# OLS 1 - diff_dose1_dose2 ~ features + dummies

model_n = '1a'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y1, X1).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.507
Model:                            OLS   Adj. R-squared:                  0.507
Method:                 Least Squares   F-statistic:                 6.728e+05
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:43:55   Log-Likelihood:            -2.6723e+07
No. Observations:             5881671   AIC:                         5.345e+07
Df Residuals:                 5881661   BIC:                         5.345e+07
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                87.0155      0.024   3605.764      0.000      86.968      87.063
Age                   1.5436      0.011    144.548      0.000       1.523       1.564
Vote diff. PSL-PT     1.1020      0.015     72.595      0.000       1.072       1.132
Avg. Income          -1.1384      0.009   -131.339      0.000      -1.155      -1.121
Female                0.3675      0.019     19.386      0.000       0.330       0.405
Black                 0.9556      0.040     23.854      0.000       0.877       1.034
Brown                 0.2895      0.024     12.275      0.000       0.243       0.336
White                -0.5586      0.024    -23.199      0.000      -0.606      -0.511
Dose1 Pfizer         -3.6227      0.025   -144.421      0.000      -3.672      -3.574
Dose1 SINOVAC       -52.1693      0.023  -2311.014      0.000     -52.214     -52.125
==============================================================================
Omnibus:                  5959382.837   Durbin-Watson:                   1.779
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        344344527.477
Skew:                           5.110   Prob(JB):                         0.00
Kurtosis:                      39.064   Cond. No.                         7.10
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [144]:
# OLS 2 - diff_dose2_dose3 ~ features + dummies

model_n = '2a'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y2, X2).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.383
Model:                            OLS   Adj. R-squared:                  0.383
Method:                 Least Squares   F-statistic:                 1.183e+05
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:43:56   Log-Likelihood:            -8.3611e+06
No. Observations:             1714620   AIC:                         1.672e+07
Df Residuals:                 1714610   BIC:                         1.672e+07
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               130.2187      0.069   1897.640      0.000     130.084     130.353
Age                   0.9156      0.031     29.753      0.000       0.855       0.976
Vote diff. PSL-PT     0.2870      0.040      7.249      0.000       0.209       0.365
Avg. Income           0.2842      0.021     13.779      0.000       0.244       0.325
Female                4.4144      0.050     88.902      0.000       4.317       4.512
Black                -0.8417      0.103     -8.143      0.000      -1.044      -0.639
Brown                -2.3058      0.064    -36.144      0.000      -2.431      -2.181
White                -0.6371      0.061    -10.496      0.000      -0.756      -0.518
Dose1 Pfizer        -10.8947      0.107   -101.976      0.000     -11.104     -10.685
Dose1 SINOVAC        47.4065      0.052    908.843      0.000      47.304      47.509
==============================================================================
Omnibus:                   260229.693   Durbin-Watson:                   1.919
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2193203.211
Skew:                          -0.482   Prob(JB):                         0.00
Kurtosis:                       8.456   Cond. No.                         9.14
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [145]:
# setting DF for OLS 1 & 2 minuns dummies for vaccines
X1 = sm.add_constant(df1[list(features) + list(dummies2)])
X2 = sm.add_constant(df2[list(features) + list(dummies2)])


In [146]:
# OLS 1 - diff_dose1_dose2 ~ features + dummies minuns dummies for vaccines

model_n = '1a2'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y1, X1).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                 1.986e+04
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:43:59   Log-Likelihood:            -2.8736e+07
No. Observations:             5881671   AIC:                         5.747e+07
Df Residuals:                 5881663   BIC:                         5.747e+07
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                71.5267      0.030   2378.641      0.000      71.468      71.586
Age                  -4.4505      0.013   -337.885      0.000      -4.476      -4.425
Vote diff. PSL-PT     1.4693      0.021     68.753      0.000       1.427       1.511
Avg. Income          -1.0313      0.012    -84.547      0.000      -1.055      -1.007
Female               -0.0904      0.027     -3.385      0.001      -0.143      -0.038
Black                -0.0717      0.056     -1.271      0.204      -0.182       0.039
Brown                -0.1450      0.033     -4.369      0.000      -0.210      -0.080
White                -1.5251      0.034    -45.000      0.000      -1.591      -1.459
==============================================================================
Omnibus:                  1376792.177   Durbin-Watson:                   0.821
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          7709361.434
Skew:                           1.015   Prob(JB):                         0.00
Kurtosis:                       8.229   Cond. No.                         6.85
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [147]:
# OLS 2 - diff_dose2_dose3 ~ features + dummies minuns dummies for vaccines

model_n = '2a2'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y2, X2).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.057
Model:                            OLS   Adj. R-squared:                  0.057
Method:                 Least Squares   F-statistic:                 1.479e+04
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:44:00   Log-Likelihood:            -8.7250e+06
No. Observations:             1714620   AIC:                         1.745e+07
Df Residuals:                 1714612   BIC:                         1.745e+07
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               144.7790      0.079   1830.582      0.000     144.624     144.934
Age                  10.4822      0.036    294.875      0.000      10.412      10.552
Vote diff. PSL-PT    -0.3993      0.049     -8.164      0.000      -0.495      -0.303
Avg. Income          -0.0169      0.025     -0.662      0.508      -0.067       0.033
Female                5.7617      0.061     93.884      0.000       5.641       5.882
Black                -2.6039      0.128    -20.377      0.000      -2.854      -2.353
Brown                -4.5398      0.079    -57.594      0.000      -4.694      -4.385
White                -1.0622      0.075    -14.152      0.000      -1.209      -0.915
==============================================================================
Omnibus:                   110301.893   Durbin-Watson:                   1.331
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           379429.080
Skew:                           0.274   Prob(JB):                         0.00
Kurtosis:                       5.238   Cond. No.                         8.61
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [148]:
# Ordered Logit - n_doses ~ features + dummies

model_n = '3a'
model_name = f"model{model_n}"

# Create the Logit model with a different name
model = OrderedModel(y3,X3,distr='logit').fit(method='bfgs', maxiter=2000)

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

Optimization terminated successfully.
         Current function value: 0.798621
         Iterations: 47
         Function evaluations: 48
         Gradient evaluations: 48


In [149]:
model3a.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:                n_doses   Log-Likelihood:            -5.5414e+06
Model:                   OrderedModel   AIC:                         1.108e+07
Method:            Maximum Likelihood   BIC:                         1.108e+07
Date:                Sun, 26 Mar 2023                                         
Time:                        15:06:33                                         
No. Observations:             6938757                                         
Df Residuals:                 6938745                                         
Df Model:                           9                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Age                   0.9481      0.001    940.510      0.000       0.946       0.950
Vote diff. PSL-PT    -0.0341      0.001    -26.387      0.000      -0.037      -0.032
Avg. Income           0.1652      0.001    217.940      0.000       0.164       0.167
Female                0.1802      0.002    112.085      0.000       0.177       0.183
Black                 0.0937      0.003     27.747      0.000       0.087       0.100
Brown                 0.1447      0.002     72.933      0.000       0.141       0.149
White                 0.2540      0.002    123.119      0.000       0.250       0.258
Dose1 Pfizer         -0.6263      0.002   -285.980      0.000      -0.631      -0.622
Dose1 SINOVAC         0.4971      0.002    247.935      0.000       0.493       0.501
1/2                  -2.1204      0.002   -907.634      0.000      -2.125      -2.116
2/3                   1.3069      0.000   2766.757      0.000       1.306       1.308
3/4                   1.5516      0.001   1342.360      0.000       1.549       1.554
=====================================================================================
"""

In [150]:
model3a.prsquared

0.16837745650190727

In [151]:
table = Stargazer([model3a, model1a, model1a2, model2a, model2a2])

In [152]:
print(table.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) \\
\hline \\[-1.8ex]
 1/2 & -2.120$^{***}$ & & & & \\
  & (0.002) & & & & \\
 2/3 & 1.307$^{***}$ & & & & \\
  & (0.000) & & & & \\
 3/4 & 1.552$^{***}$ & & & & \\
  & (0.001) & & & & \\
 Age & 0.948$^{***}$ & 1.544$^{***}$ & -4.450$^{***}$ & 0.916$^{***}$ & 10.482$^{***}$ \\
  & (0.001) & (0.011) & (0.013) & (0.031) & (0.036) \\
 Avg. Income & 0.165$^{***}$ & -1.138$^{***}$ & -1.031$^{***}$ & 0.284$^{***}$ & -0.017$^{}$ \\
  & (0.001) & (0.009) & (0.012) & (0.021) & (0.025) \\
 Black & 0.094$^{***}$ & 0.956$^{***}$ & -0.072$^{}$ & -0.842$^{***}$ & -2.604$^{***}$ \\
  & (0.003) & (0.040) & (0.056) & (0.103) & (0.128) \\
 Brown & 0.145$^{***}$ & 0.289$^{***}$ & -0.145$^{***}$ & -2.306$^{***}$ & -4.540$^{***}$ \\
  & (0.002) & (0.024) & (0.033) & (0.064) & (0.079) \\
 Dose1 Pfizer & -0.626$^{***}$ & -3.623$^{***}$ & & -10.895$^{***}$ & \\

In [153]:
# Defining population

group = 'nome_regiao'
region = 'Sul'
vaccine = 'dose1_JANSSEN'

df1 = df[df[target[0]] > 0]
df2 = df[df[target[1]] > 0]
df1 = df1.dropna(how='any').copy()
df2 = df2.dropna(how='any').copy()

df3 = df[(df[vaccine] == 0) & (df[group] == region)]
df1 = df1[(df1[vaccine] == 0) & (df1[group] == region)]
df2 = df2[(df2[vaccine] == 0) & (df2[group] == region)]

# setting DF for OLS 1 & 2
X1 = sm.add_constant(df1[list(features) + list(dummies)])
X2 = sm.add_constant(df2[list(features) + list(dummies)])

# setting DF for logit (X3)
X3 = df3[list(features)+ list(dummies)]

# target1 (y1) as diff_dose1_dose2
y1 = df1[target[0]].values.reshape(-1, 1)

# target2 (y2) as diff_dose2_dose3
y2 = df2[target[1]].values.reshape(-1, 1)

# and target3 (y3) as n_doses
y3 = df3[target[2]]

In [154]:
# OLS 1 - diff_dose1_dose2 ~ features + dummies

model_n = '1a'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y1, X1).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.504
Model:                            OLS   Adj. R-squared:                  0.504
Method:                 Least Squares   F-statistic:                 2.702e+05
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        15:06:42   Log-Likelihood:            -1.0763e+07
No. Observations:             2395819   AIC:                         2.153e+07
Df Residuals:                 2395809   BIC:                         2.153e+07
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                85.8707      0.036   2402.270      0.000      85.801      85.941
Age                   1.6566      0.016    103.179      0.000       1.625       1.688
Vote diff. PSL-PT     0.2523      0.022     11.703      0.000       0.210       0.294
Avg. Income          -1.2801      0.017    -76.543      0.000      -1.313      -1.247
Female               -0.5304      0.028    -18.777      0.000      -0.586      -0.475
Black                 2.8878      0.072     40.306      0.000       2.747       3.028
Brown                 2.5415      0.077     33.117      0.000       2.391       2.692
White                 1.0922      0.031     35.303      0.000       1.032       1.153
Dose1 Pfizer        -11.4661      0.036   -317.372      0.000     -11.537     -11.395
Dose1 SINOVAC       -52.7255      0.035  -1516.037      0.000     -52.794     -52.657
==============================================================================
Omnibus:                  2221285.869   Durbin-Watson:                   1.918
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        130832979.819
Skew:                           4.375   Prob(JB):                         0.00
Kurtosis:                      38.129   Cond. No.                         9.38
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [155]:
# OLS 2 - diff_dose2_dose3 ~ features + dummies

model_n = '2a'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y2, X2).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.399
Model:                            OLS   Adj. R-squared:                  0.399
Method:                 Least Squares   F-statistic:                 5.510e+04
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        15:06:43   Log-Likelihood:            -3.5977e+06
No. Observations:              748358   AIC:                         7.195e+06
Df Residuals:                  748348   BIC:                         7.195e+06
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               144.7436      0.095   1531.406      0.000     144.558     144.929
Age                   3.0070      0.041     72.469      0.000       2.926       3.088
Vote diff. PSL-PT    -2.8332      0.053    -53.203      0.000      -2.938      -2.729
Avg. Income           0.4816      0.039     12.473      0.000       0.406       0.557
Female                4.3849      0.071     62.114      0.000       4.247       4.523
Black                 4.3116      0.187     23.112      0.000       3.946       4.677
Brown                 2.0417      0.216      9.462      0.000       1.619       2.465
White                 1.0501      0.075     13.961      0.000       0.903       1.198
Dose1 Pfizer        -13.3828      0.125   -106.831      0.000     -13.628     -13.137
Dose1 SINOVAC        42.1439      0.075    559.108      0.000      41.996      42.292
==============================================================================
Omnibus:                   109789.369   Durbin-Watson:                   1.946
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           669968.815
Skew:                          -0.563   Prob(JB):                         0.00
Kurtosis:                       7.496   Cond. No.                         12.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [156]:
# setting DF for OLS 1 & 2 minuns dummies for vaccines
X1 = sm.add_constant(df1[list(features) + list(dummies2)])
X2 = sm.add_constant(df2[list(features) + list(dummies2)])


In [157]:
# OLS 1 - diff_dose1_dose2 ~ features + dummies minuns dummies for vaccines

model_n = '1a2'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y1, X1).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     5597.
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        15:06:44   Log-Likelihood:            -1.1583e+07
No. Observations:             2395819   AIC:                         2.317e+07
Df Residuals:                 2395811   BIC:                         2.317e+07
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                69.9387      0.043   1616.903      0.000      69.854      70.023
Age                  -3.1624      0.020   -159.624      0.000      -3.201      -3.124
Vote diff. PSL-PT     0.8732      0.030     28.786      0.000       0.814       0.933
Avg. Income          -1.7740      0.024    -75.355      0.000      -1.820      -1.728
Female               -1.6137      0.040    -40.590      0.000      -1.692      -1.536
Black                 1.5327      0.101     15.194      0.000       1.335       1.730
Brown                 0.8442      0.108      7.814      0.000       0.632       1.056
White                -0.5270      0.044    -12.106      0.000      -0.612      -0.442
==============================================================================
Omnibus:                   495294.316   Durbin-Watson:                   0.919
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2585043.459
Skew:                           0.905   Prob(JB):                         0.00
Kurtosis:                       7.756   Cond. No.                         9.06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [158]:
# OLS 2 - diff_dose2_dose3 ~ features + dummies minuns dummies for vaccines

model_n = '2a2'
model_name = f"model{model_n}"

# Create the OLS model with a different name
model = sm.OLS(y2, X2).fit()

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.084
Model:                            OLS   Adj. R-squared:                  0.084
Method:                 Least Squares   F-statistic:                     9864.
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        15:06:44   Log-Likelihood:            -3.7549e+06
No. Observations:              748358   AIC:                         7.510e+06
Df Residuals:                  748350   BIC:                         7.510e+06
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               153.9673      0.107   1432.614      0.000     153.757     154.178
Age                  11.3623      0.048    236.554      0.000      11.268      11.456
Vote diff. PSL-PT    -5.1717      0.066    -78.914      0.000      -5.300      -5.043
Avg. Income           0.9592      0.048     20.141      0.000       0.866       1.053
Female                6.1302      0.087     70.437      0.000       5.960       6.301
Black                 6.2640      0.230     27.219      0.000       5.813       6.715
Brown                 4.0893      0.266     15.362      0.000       3.568       4.611
White                 3.1619      0.093     34.105      0.000       2.980       3.344
==============================================================================
Omnibus:                    43907.973   Durbin-Watson:                   1.358
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           193274.544
Skew:                          -0.038   Prob(JB):                         0.00
Kurtosis:                       5.488   Cond. No.                         12.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [159]:
# Ordered Logit - n_doses ~ features + dummies

model_n = '3a'
model_name = f"model{model_n}"

# Create the Logit model with a different name
model = OrderedModel(y3,X3,distr='logit').fit(method='bfgs', maxiter=2000)

# Set the name of the variable dynamically using exec
exec(f"{model_name} = model")

# Add the model name to the list of models
models.append(model_name)

Optimization terminated successfully.
         Current function value: 0.747839
         Iterations: 45
         Function evaluations: 46
         Gradient evaluations: 46


In [160]:
model3a.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:                n_doses   Log-Likelihood:            -2.0299e+06
Model:                   OrderedModel   AIC:                         4.060e+06
Method:            Maximum Likelihood   BIC:                         4.060e+06
Date:                Sun, 26 Mar 2023                                         
Time:                        15:14:21                                         
No. Observations:             2714418                                         
Df Residuals:                 2714406                                         
Df Model:                           9                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Age                   0.9526      0.002    573.236      0.000       0.949       0.956
Vote diff. PSL-PT     0.0099      0.002      4.957      0.000       0.006       0.014
Avg. Income           0.1847      0.002    115.823      0.000       0.182       0.188
Female                0.2836      0.003    107.348      0.000       0.278       0.289
Black                -0.2607      0.007    -39.841      0.000      -0.274      -0.248
Brown                -0.3753      0.007    -54.591      0.000      -0.389      -0.362
White                -0.1052      0.003    -36.066      0.000      -0.111      -0.099
Dose1 Pfizer         -0.5305      0.004   -150.012      0.000      -0.537      -0.524
Dose1 SINOVAC         0.8035      0.003    237.468      0.000       0.797       0.810
1/2                  -2.4151      0.004   -612.505      0.000      -2.423      -2.407
2/3                   1.3611      0.001   1793.572      0.000       1.360       1.363
3/4                   1.8037      0.002    734.459      0.000       1.799       1.809
=====================================================================================
"""

In [161]:
model3a.prsquared

0.18637977401152017

In [162]:
table = Stargazer([model3a, model1a, model1a2, model2a, model2a2])

In [163]:
print(table.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) \\
\hline \\[-1.8ex]
 1/2 & -2.415$^{***}$ & & & & \\
  & (0.004) & & & & \\
 2/3 & 1.361$^{***}$ & & & & \\
  & (0.001) & & & & \\
 3/4 & 1.804$^{***}$ & & & & \\
  & (0.002) & & & & \\
 Age & 0.953$^{***}$ & 1.657$^{***}$ & -3.162$^{***}$ & 3.007$^{***}$ & 11.362$^{***}$ \\
  & (0.002) & (0.016) & (0.020) & (0.041) & (0.048) \\
 Avg. Income & 0.185$^{***}$ & -1.280$^{***}$ & -1.774$^{***}$ & 0.482$^{***}$ & 0.959$^{***}$ \\
  & (0.002) & (0.017) & (0.024) & (0.039) & (0.048) \\
 Black & -0.261$^{***}$ & 2.888$^{***}$ & 1.533$^{***}$ & 4.312$^{***}$ & 6.264$^{***}$ \\
  & (0.007) & (0.072) & (0.101) & (0.187) & (0.230) \\
 Brown & -0.375$^{***}$ & 2.541$^{***}$ & 0.844$^{***}$ & 2.042$^{***}$ & 4.089$^{***}$ \\
  & (0.007) & (0.077) & (0.108) & (0.216) & (0.266) \\
 Dose1 Pfizer & -0.531$^{***}$ & -11.466$^{***}$ & & -13.383$^{***}$ & 